# Automatic lab note generation

This notebook demonstrates an automatic lab note generation pipline using Google GenAI

Converting videos-to-lab notes involves following steps: 

## Protocol finder:
1. Video_analyzer_agent: Analyzes video to get the laboratory procedure that can be compared with protocols on Confluence.
2. lab_knowledge_agent: Select protocol name which best captures the actions being performed in the video

## Lab note generator:
3. Video comparing to ground-of-truth protocol → lab documentation + errors in procedure
4. Analytics based on benchmark dataset: automatic comparison of errors found by documentation assistent vs actual errors

This notebook contains example code for the main functions of all these steps.


In [ ]:
from __future__ import annotations

%reload_ext autoreload
%autoreload 2

import sys
from pathlib import Path

path_to_append = Path(Path.cwd()).parent
sys.path.append(str(path_to_append))
from proteomics_specialist.sub_agents.video_analyzer_agent import agent

In [ ]:
# Step 2: Analyzing video to get the laboratory procedure

test_cases = [
    # 'Video path: /Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/DisconnectingColumn_docuCorrect.MP4. Analyze the video.',
    # "Analyse this video: \"/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/DisconnectingColumn_docuCorrect.MP4\".",
    # 'Analyse this video: "gs://ai-proteomics-advisor/input_video/ConnectingColumnSampleLine_docuWithoutStandbyANDtimsControl.mp4". What is happening?',
    # 'Analyse this video: "gs://ai-proteomics-advisor/input_video/ConnectingColumnSampleLine_docuWithoutStandbyANDtimsControl.mp4".',
    "gs://ai-proteomics-advisor/lab_note_generation_examples/ConnectingColumnSampleLine_docuWithoutStandbyANDtimsControl.mp4",
    # 'Analyse this video: "/Users/patriciaskowronek/Downloads/ultra_short.mp4".. What is happening?',
    # "'/Users/patriciaskowronek/Downloads/ultra_short.mp4"."',
    # "/Users/patriciaskowronek/Downloads/ultra_short.mp4".',
    # "Check this document.pdf for errors.",
    # "Analyse this video: . what is happening.",
]

for test in test_cases:
    result = agent.analyze_proteomics_video(test)
    print(result)
    print()

In [ ]:
from IPython.display import Markdown

from proteomics_specialist.sub_agents.lab_note_generator_agent import agent

result = agent.generate_lab_notes(
    query="gs://ai-proteomics-advisor/lab_note_generation_examples/ConnectingColumnSampleLine_docuWithoutStandbyANDtimsControl.mp4",
    tool_context=None,
    protocol_input="""
    # Connecting IonOpticks Column & sample line of Evosep

    ## Abstract
    This protocol explains how to connect an IonOpticks column inserted into a timsTOF Ultra with an sample line of an Evosep.


    ## Materials

    ### Equipment
    - IonOpticks Column
    - timsTOF Ultra Mass Spectrometer
      - Equipped with UltraSource ion source
      - Equipped with column oven for temperature control
    - Evosep One LC System
      - with sample line
    - NanoViper Adapter (black)
    - Pliers


    ## Procedure
    *Estimated timing: 3 minutes*

    1. Verfy that the instrument is in standing by mode. If not press the on/off button to switch from operate mode to standing by mode.

    ### Connect column and sample line
    2. Verify that an IonOpticks column is already inserted into the UltraSource of the timsTOF Ultra.
    3. Attach a black NanoViper adapter to the LC sample line.
    4. Remove any access liquid at the top of the nanoViper of the sample line for instance by snipping it off.
    5. Hold the column fititng of the IonOpticks column with a pliers.
    6. Hand-tighten the NanoViper of the LC sample line with the column fitting (Figure 1).
      CRITICAL STEP: Be careful not to overtighten the connection between the IonOpticks column and the timsTOF Ultra. Otherwise you can damage the column and the LC sample line.
    7. Remove the NanoViper adapter so that the oven can be tightly closed.
    8. Draw the oven closer to the UltraSource, secure it with the screw on the bottom of the oven (Figure 2).
    9. Lift and place the metal grounding screw at the column-sample line connection to establish proper ESI spray grounding with the column oven.
    10. Close the lid of the oven.
    11. Verify the temperature at 50°C for IonOpticks columns, as indicated by three illuminated LEDs on the column oven, respectively.

    ### Switch timsTOF to operate and idle flow
    12. In timsControl, transition the instrument to the operation mode by using the on/off button.
    13. Navigate to Hystar and ensure that the idle flow is active. If not right-click on the Evosep logo, choosing 'preparation', and then selecting 'idle flow' and 'Run'.
    14. Return to timsControl. Check the MS signal. It should be around 2.5x10^7.


    ## Expected Results
    - In timsControl, signal intensity should be above 10^7
    - Stable signal in in timsControl (in the windows for Mobilogram (timsView) and mass spectrum)


    ## Figures
    ### Figure 1: Connection of a sample line of a LC system with a separation column and emitter.**

    ### Figure 2: UltraSource.**


    ## References
    1. Skowronek, P., Wallmann, G., Wahle, M. et al. An accessible workflow for high-sensitivity proteomics using parallel accumulation-serial fragmentation (PASEF). Nat Protoc (2025). https://doi.org/10.1038/s41596-024-01104-w
    """,
)
Markdown(result["lab_notes"])

In [ ]:
# Uploading a single file to gcs
import os

from google.cloud import storage

from proteomics_specialist.sub_agents import utils

bucket_name = os.getenv("GOOGLE_CLOUD_STORAGE_BUCKET")
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)

path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/TimsCalibration_docuSavedMethod.mov"
file_path, file_uri, filename, blob = utils.upload_file_from_path_to_gcs(
    path, bucket, "input_for_lab_note"
)
print(file_path, file_uri, filename, blob.metadata)

In [ ]:
# Uploading a folder to gcs
import os

from google.cloud import storage

from proteomics_specialist.sub_agents import utils

bucket_name = os.getenv("GOOGLE_CLOUD_STORAGE_BUCKET")
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)

folder_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation"
file_paths = []
for root, _dirs, files in os.walk(folder_path):
    for file in files:
        path = Path(root) / file
        file_path, file_uri, filename, blob = utils.upload_file_from_path_to_gcs(
            path, bucket, "input_for_lab_note"
        )
        print(file_path, file_uri, filename, blob.metadata)

In [ ]:
# Generting the benchmark_data.csv
# ruff: noqa: RUF001

import pandas as pd

benchmark_dataset_path = "/Users/patriciaskowronek/Documents/proteomics_specialist/eval/eval_lab_note_generation/benchmark_data.csv"

df_benchmark_data = pd.read_csv(benchmark_dataset_path, index_col=0)
list_new_rows = [
    {
        "eval_set_name": "PlaceEvotips_docuCorrect",
        "protocol": """The page titled "Placing Evotips in Evotip Boxes on the Evosep One System" contains following information: - ## Abstract\nThis protocol describes the proper procedure for inspecting Evotips and placing Evotips in Evotip boxes on the liquid chromatography system Evosep One.\n\n\n## Materials\n\n### Equipment\n- Evotips\n  - Single-use stage tips for sample injection\n  - Rack layout: Two columns (left and right)\n  - Left column (top to bottom): S1, S2, S3\n  - Right column (top to bottom): S4, S5, S6\n  - Within each box: Standard 96-well format with A1 (top left), A12 (top right), H12 (bottom right)\n- Evotip Boxes\n  - 96-well format (A1-H12) (Figure 1)\n- Evosep One System\n  - Liquid chromatography system\n\n### Reagents\n- Formic acid (FA)\n  ! CAUTION: This liquid may be corrosive. It is harmful and can cause damage if direct contact occurs.\n\n### Reagent setup\n- Buffer A: Consists of 0.1% (vol/vol) FA. The buffers are stable for at least 6 months at room temperature as long as they are protected from sunlight.\n\n\n## Procedure\n*Estimated timing: less than 1 minute*\n\n1. Visually inspect the Evotip box intended for S1 to verify it's filled with Buffer A solution to a minimum depth of 1 cm. Verification of the Evotip box placed at S3 is not necessary as it will only contain blanks.\n2. Place Evotip Box at S1 within the rack system of the Evosep instrument. Ensure each box is firmly seated in its designated position.\n3. Place an empty Evotip Box for Blank tips at S3. Ensure each box is firmly seated in its designated position.\n4. Inspect each Evotip before placement to verify its condition. Properly prepared Evotips should display a pale-colored SPE material disc with visible solvent above it (Figure 2).\n   **CRITICAL STEP**: Discard any Evotips showing signs of dryness or displaying a white-colored disc, as these conditions indicate compromised functionality that could affect sample analysis.\n5. Place the verified Evotips into the prepared Evotip boxes at S1 from A1 to A6.\n6. Place empty Evotips, called Blanks, at S3 from A1 to A6.\n7. Document the precise position of each placed Evotip.\n\n\n## Expected Results\nWhen the procedure is performed correctly, you should observe:\n- Properly seated Evotip boxes in the rack system\n- Visible Buffer A solution in boxes (1 cm depth)\n- All non-blank Evotips showing pale-colored SPE material discs & clear solvent meniscus above each SPE disc of each Evotip\n- Accurate documentation of tip positions: Evotips that are placed at S1 from A1 to A6 and blanks placed at S3 from A1 to A6.\n\n\n## Figures\n\n### Figure 1: Evosep positions\n[Placeholder for close-up of single Evotip box showing well positions (A1-H12)]\n\n### Figure 2: Evotip Quality Assessment\n- Most Evotips: Properly hydrated Evotip with pale-colored disc and visible solvent\n- Orange-highlighted Evotip: Compromised Evotip showing white/dry disc\n\n\n## References\n1. Evosep One - User Guide: https://www.evosep.com/wp-content/uploads/2024/06/Evosep-One-User-Guide-v18.pdf\n2. Sample loading protocol for Evotips: https://www.evosep.com/wp-content/uploads/2020/03/Sample-loading-protocol.pdf\n""",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/PlaceEvotips_docuCorrect.MP4""",
        "recording_type": "camera",
        "ground_truth_lab_notes": "# Placing Evotips in Evotip Boxes on the Evosep One System\n\n## Aim\nPlacing Evotips in Evotip boxes: Evotips with HeLa at S1 from A1 to A6 and blanks placed at S3 from A1 to A6.\n\n\n## Materials\n\n### Equipment\n- Evotips\n- Evotip Boxes\n- Evosep One System\n\n### Reagent setup\n- Buffer A:0.1% (vol/vol) FA\n\n\n## Procedure\n*Timing: less than 1 minute*\n\n1. The Evotip box intended for S1 was verified to be filled with Buffer A solution by visual inspection. Verification was omitted for the box placed at S3 since it will only contain blanks.\n2. Placed Evotip Box at S1 within the rack system of the Evosep instrument. Ensured box is firmly seated in its designated position.\n3. Placed an empty Evotip Box for Blank tips at S3. Ensured box is firmly seated in its designated position.\n4. Inspected each Evotip before placement to verify its condition. All Evotips were properly prepared as they display a pale-colored SPE material disc with visible solvent above it.\n5. Placed the verified Evotips into the prepared Evotip boxes at S1 from A1 to A12.\n6. Placed empty Evotips, called Blanks, at S3 from A1 to A12.\n7. ❌ Omitted: Did not document the precise position of each placed Evotip during the recorded procedure.\n\n\n## Results\n- Properly seated Evotip boxes in the rack system\n- Visible Buffer A solution in boxes (1 cm depth)\n- All non-blank Evotips showing pale-colored SPE material discs & clear solvent meniscus above each SPE disc of each Evotip\n- Evotips that are placed at S1 from A1 to A6.\n- Blanks placed at S3 from A1 to A6.\n",
        "error_dict": """[{"Step": 1.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 2.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 3.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 4.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 5.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 6.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 7.0, "Benchmark": "Error", "Class": "Omitted", "Skill": "GeneralKnowledge"}]""",
        "comments": "",
    },
    {
        "eval_set_name": "PlaceEvotips_docuWrongPosition",
        "protocol": """The page titled "Placing Evotips in Evotip Boxes on the Evosep One System" contains following information: - ## Abstract\nThis protocol describes the proper procedure for inspecting Evotips and placing Evotips in Evotip boxes on the liquid chromatography system Evosep One.\n\n\n## Materials\n\n### Equipment\n- Evotips\n  - Single-use stage tips for sample injection\n  - Rack layout: Two columns (left and right)\n  - Left column (top to bottom): S1, S2, S3\n  - Right column (top to bottom): S4, S5, S6\n  - Within each box: Standard 96-well format with A1 (top left), A12 (top right), H12 (bottom right)\n- Evotip Boxes\n  - 96-well format (A1-H12) (Figure 1)\n- Evosep One System\n  - Liquid chromatography system\n\n### Reagents\n- Formic acid (FA)\n  ! CAUTION: This liquid may be corrosive. It is harmful and can cause damage if direct contact occurs.\n\n### Reagent setup\n- Buffer A: Consists of 0.1% (vol/vol) FA. The buffers are stable for at least 6 months at room temperature as long as they are protected from sunlight.\n\n\n## Procedure\n*Estimated timing: less than 1 minute*\n\n1. Visually inspect the Evotip box intended for S1 to verify it's filled with Buffer A solution to a minimum depth of 1 cm. Verification of the Evotip box placed at S3 is not necessary as it will only contain blanks.\n2. Place Evotip Box at S1 within the rack system of the Evosep instrument. Ensure each box is firmly seated in its designated position.\n3. Place an empty Evotip Box for Blank tips at S3. Ensure each box is firmly seated in its designated position.\n4. Inspect each Evotip before placement to verify its condition. Properly prepared Evotips should display a pale-colored SPE material disc with visible solvent above it (Figure 2).\n   **CRITICAL STEP**: Discard any Evotips showing signs of dryness or displaying a white-colored disc, as these conditions indicate compromised functionality that could affect sample analysis.\n5. Place the verified Evotips into the prepared Evotip boxes at S1 from A1 to A6.\n6. Place empty Evotips, called Blanks, at S3 from A1 to A6.\n7. Document the precise position of each placed Evotip.\n\n\n## Expected Results\nWhen the procedure is performed correctly, you should observe:\n- Properly seated Evotip boxes in the rack system\n- Visible Buffer A solution in boxes (1 cm depth)\n- All non-blank Evotips showing pale-colored SPE material discs & clear solvent meniscus above each SPE disc of each Evotip\n- Accurate documentation of tip positions: Evotips that are placed at S1 from A1 to A6 and blanks placed at S3 from A1 to A6.\n\n\n## Figures\n\n### Figure 1: Evosep positions\n[Placeholder for close-up of single Evotip box showing well positions (A1-H12)]\n\n### Figure 2: Evotip Quality Assessment\n- Most Evotips: Properly hydrated Evotip with pale-colored disc and visible solvent\n- Orange-highlighted Evotip: Compromised Evotip showing white/dry disc\n\n\n## References\n1. Evosep One - User Guide: https://www.evosep.com/wp-content/uploads/2024/06/Evosep-One-User-Guide-v18.pdf\n2. Sample loading protocol for Evotips: https://www.evosep.com/wp-content/uploads/2020/03/Sample-loading-protocol.pdf\n""",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/PlaceEvotips_docuWrongPosition.MP4""",
        "recording_type": "camera",
        "ground_truth_lab_notes": "# Placing Evotips in Evotip Boxes on the Evosep One System\n\n## Aim\nPlacing Evotips in Evotip boxes: Evotips with HeLa at S1 from A1 to A6 and blanks placed at S3 from A1 to A6.\n\n\n## Materials\n\n### Equipment\n- Evotips\n- Evotip Boxes\n- Evosep One System\n\n### Reagent setup\n- Buffer A:0.1% (vol/vol) FA\n\n\n## Procedure\n*Timing: less than 1 minute*\n\n1. The Evotip box intended for S1 was verified to be filled with Buffer A solution by visual inspection. Verification was omitted for the box placed at S3 since it will only contain blanks.\n2. Placed Evotip Box at S1 within the rack system of the Evosep instrument. Ensured box is firmly seated in its designated position.\n3. Place an empty Evotip Box for Blank tips at S3. Ensured box is firmly seated in its designated position.\n4. ❌ **Omitted:** Inspected only some of the Evotips before placement to verify its condition. Properly prepared Evotips should display a pale-colored SPE material disc with visible solvent above it. All Evotips were fine.\n5. ❌ **Error:** Placed the verified Evotips into the prepared Evotip boxes at S1, but positioned them from B1 to B3 and B5 to B7.\n6. Place empty Evotips, called Blanks, at S3 from A1 to A12.\n7. ❌ **Omitted:** Did not document the precise position of each placed Evotip during the recorded procedure.\n\n\n## Results\n- Properly seated Evotip boxes in the rack system\n- Visible Buffer A solution in boxes (1 cm depth)\n- All non-blank Evotips showing pale-colored SPE material discs & clear solvent meniscus above each SPE disc of each Evotip\n- Evotips that are placed at S1 from B1 to B3 and B5 to B7.\n- Blanks placed at S3 from A1 to A6.\n",
        "error_dict": """[{"Step": 1.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 2.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 3.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 4.0, "Benchmark": "Error", "Class": "Omitted", "Skill": "Fast"},{"Step": 5.0, "Benchmark": "Error", "Class": "Error", "Skill": "SpatialOrientation"},{"Step": 6.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 7.0, "Benchmark": "Error", "Class": "Omitted", "Skill": "GeneralKnowledge"}]""",
        "comments": "Step 4: Evotip placed at wrong position",
    },
    {
        "eval_set_name": "PlaceEvotips_docuLiquidNotChecked",
        "protocol": """The page titled "Placing Evotips in Evotip Boxes on the Evosep One System" contains following information: - ## Abstract\nThis protocol describes the proper procedure for inspecting Evotips and placing Evotips in Evotip boxes on the liquid chromatography system Evosep One.\n\n\n## Materials\n\n### Equipment\n- Evotips\n  - Single-use stage tips for sample injection\n  - Rack layout: Two columns (left and right)\n  - Left column (top to bottom): S1, S2, S3\n  - Right column (top to bottom): S4, S5, S6\n  - Within each box: Standard 96-well format with A1 (top left), A12 (top right), H12 (bottom right)\n- Evotip Boxes\n  - 96-well format (A1-H12) (Figure 1)\n- Evosep One System\n  - Liquid chromatography system\n\n### Reagents\n- Formic acid (FA)\n  ! CAUTION: This liquid may be corrosive. It is harmful and can cause damage if direct contact occurs.\n\n### Reagent setup\n- Buffer A: Consists of 0.1% (vol/vol) FA. The buffers are stable for at least 6 months at room temperature as long as they are protected from sunlight.\n\n\n## Procedure\n*Estimated timing: less than 1 minute*\n\n1. Visually inspect the Evotip box intended for S1 to verify it's filled with Buffer A solution to a minimum depth of 1 cm. Verification of the Evotip box placed at S3 is not necessary as it will only contain blanks.\n2. Place Evotip Box at S1 within the rack system of the Evosep instrument. Ensure each box is firmly seated in its designated position.\n3. Place an empty Evotip Box for Blank tips at S3. Ensure each box is firmly seated in its designated position.\n4. Inspect each Evotip before placement to verify its condition. Properly prepared Evotips should display a pale-colored SPE material disc with visible solvent above it (Figure 2).\n   **CRITICAL STEP**: Discard any Evotips showing signs of dryness or displaying a white-colored disc, as these conditions indicate compromised functionality that could affect sample analysis.\n5. Place the verified Evotips into the prepared Evotip boxes at S1 from A1 to A6.\n6. Place empty Evotips, called Blanks, at S3 from A1 to A6.\n7. Document the precise position of each placed Evotip.\n\n\n## Expected Results\nWhen the procedure is performed correctly, you should observe:\n- Properly seated Evotip boxes in the rack system\n- Visible Buffer A solution in boxes (1 cm depth)\n- All non-blank Evotips showing pale-colored SPE material discs & clear solvent meniscus above each SPE disc of each Evotip\n- Accurate documentation of tip positions: Evotips that are placed at S1 from A1 to A6 and blanks placed at S3 from A1 to A6.\n\n\n## Figures\n\n### Figure 1: Evosep positions\n[Placeholder for close-up of single Evotip box showing well positions (A1-H12)]\n\n### Figure 2: Evotip Quality Assessment\n- Most Evotips: Properly hydrated Evotip with pale-colored disc and visible solvent\n- Orange-highlighted Evotip: Compromised Evotip showing white/dry disc\n\n\n## References\n1. Evosep One - User Guide: https://www.evosep.com/wp-content/uploads/2024/06/Evosep-One-User-Guide-v18.pdf\n2. Sample loading protocol for Evotips: https://www.evosep.com/wp-content/uploads/2020/03/Sample-loading-protocol.pdf\n""",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/PlaceEvotips_docuLiquidNotChecked.MP4""",
        "recording_type": "camera",
        "ground_truth_lab_notes": "# Placing Evotips in Evotip Boxes on the Evosep One System\n\n## Aim\nPlacing Evotips in Evotip boxes: Evotips with HeLa at S1 from A1 to A6 and blanks placed at S3 from A1 to A6.\n\n\n## Materials\n\n### Equipment\n- Evotips\n- Evotip Boxes\n- Evosep One System\n\n### Reagent setup\n- Buffer A:0.1% (vol/vol) FA\n\n## Procedure\n*Timing: less than 1 minute*\n\n1. ❌ **Omitted:** Did not verify that Evotip box intended for S1 or S3 is filled to a minimum depth of 1 cm with Buffer A solution by visual inspection.\n2. Placed Evotip Box at S1 within the rack system of the Evosep instrument. Ensured box is firmly seated in its designated position.\n3. Placed an empty Evotip Box for Blank tips at S3. Ensured box is firmly seated in its designated position.\n4. ❌ **Omitted:** Inspected only half of the Evotip before placement to verify its condition. This half was properly prepared as they display a pale-colored SPE material disc with visible solvent above it.\n5. Placed the verified Evotips into the prepared Evotip boxes at S1 from A1 to A6.\n6. Placed empty Evotips, called Blanks, at S3 from A1 to A6.\n7. ❌ **Omitted:** Did not document the precise position of each placed Evotip during the recorded procedure.\n\n\n## Results\n- Properly seated Evotip boxes in the rack system\n- Did not check for visible Buffer A solution in boxes (1 cm depth)\n- Verified half of the non-blank Evotips that showes pale-colored SPE material discs & clear solvent meniscus above each SPE disc of each Evotip.\n- Evotips that are placed at S1 from A1 to A6 and blanks placed at S3 from A1 to A6.\n",
        "error_dict": """[
  {"Step": 1, "Benchmark": "Error", "Class": "Omitted", "Skill": "Fast"},
  {"Step": 2, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 3, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 4, "Benchmark": "Error", "Class": "Omitted", "Skill": "Fast"},
  {"Step": 5, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 6, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 7, "Benchmark": "Error", "Class": "Omitted", "Skill": "GeneralKnowledge"}
]""",
        "comments": "Step 1: Liquid not checked, Step 3: Evotip not checked",
    },
    {
        "eval_set_name": "PlaceEvotips_docuBoxAngeled",
        "protocol": """The page titled "Placing Evotips in Evotip Boxes on the Evosep One System" contains following information: - ## Abstract\nThis protocol describes the proper procedure for inspecting Evotips and placing Evotips in Evotip boxes on the liquid chromatography system Evosep One.\n\n\n## Materials\n\n### Equipment\n- Evotips\n  - Single-use stage tips for sample injection\n  - Rack layout: Two columns (left and right)\n  - Left column (top to bottom): S1, S2, S3\n  - Right column (top to bottom): S4, S5, S6\n  - Within each box: Standard 96-well format with A1 (top left), A12 (top right), H12 (bottom right)\n- Evotip Boxes\n  - 96-well format (A1-H12) (Figure 1)\n- Evosep One System\n  - Liquid chromatography system\n\n### Reagents\n- Formic acid (FA)\n  ! CAUTION: This liquid may be corrosive. It is harmful and can cause damage if direct contact occurs.\n\n### Reagent setup\n- Buffer A: Consists of 0.1% (vol/vol) FA. The buffers are stable for at least 6 months at room temperature as long as they are protected from sunlight.\n\n\n## Procedure\n*Estimated timing: less than 1 minute*\n\n1. Visually inspect the Evotip box intended for S1 to verify it's filled with Buffer A solution to a minimum depth of 1 cm. Verification of the Evotip box placed at S3 is not necessary as it will only contain blanks.\n2. Place Evotip Box at S1 within the rack system of the Evosep instrument. Ensure each box is firmly seated in its designated position.\n3. Place an empty Evotip Box for Blank tips at S3. Ensure each box is firmly seated in its designated position.\n4. Inspect each Evotip before placement to verify its condition. Properly prepared Evotips should display a pale-colored SPE material disc with visible solvent above it (Figure 2).\n   **CRITICAL STEP**: Discard any Evotips showing signs of dryness or displaying a white-colored disc, as these conditions indicate compromised functionality that could affect sample analysis.\n5. Place the verified Evotips into the prepared Evotip boxes at S1 from A1 to A6.\n6. Place empty Evotips, called Blanks, at S3 from A1 to A6.\n7. Document the precise position of each placed Evotip.\n\n\n## Expected Results\nWhen the procedure is performed correctly, you should observe:\n- Properly seated Evotip boxes in the rack system\n- Visible Buffer A solution in boxes (1 cm depth)\n- All non-blank Evotips showing pale-colored SPE material discs & clear solvent meniscus above each SPE disc of each Evotip\n- Accurate documentation of tip positions: Evotips that are placed at S1 from A1 to A6 and blanks placed at S3 from A1 to A6.\n\n\n## Figures\n\n### Figure 1: Evosep positions\n[Placeholder for close-up of single Evotip box showing well positions (A1-H12)]\n\n### Figure 2: Evotip Quality Assessment\n- Most Evotips: Properly hydrated Evotip with pale-colored disc and visible solvent\n- Orange-highlighted Evotip: Compromised Evotip showing white/dry disc\n\n\n## References\n1. Evosep One - User Guide: https://www.evosep.com/wp-content/uploads/2024/06/Evosep-One-User-Guide-v18.pdf\n2. Sample loading protocol for Evotips: https://www.evosep.com/wp-content/uploads/2020/03/Sample-loading-protocol.pdf\n""",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/PlaceEvotips_docuBoxAngeled.MP4""",
        "recording_type": "camera",
        "ground_truth_lab_notes": "# Placing Evotips in Evotip Boxes on the Evosep One System\n\n## Aim\nPlacing Evotips in Evotip boxes: Evotips with HeLa at S1 from A1 to A6 and blanks placed at S3 from A1 to A6.\n\n\n## Materials\n\n### Equipment\n- Evotips\n- Evotip Boxes\n- Evosep One System\n\n### Reagent setup\n- Buffer A:0.1% (vol/vol) FA\n\n\n## Procedure\n*Timing: less than 1 minute*\n\n1. The Evotip box intended for S1 was verified to be filled with Buffer A solution by visual inspection. Verification was omitted for the box placed at S3 since it will only contain blanks.\n2. ❌ **Error:** Placed Evotip Box at S1 within the rack system of the Evosep instrument, but did not ensure that box is firmly seated in its designated position. In fact, box is not properly seated and placed at an angle on the rack.\n3. Placed an empty Evotip Box for Blank tips at S3. Ensured box is firmly seated in its designated position.\n4. Inspected each Evotip before placement to verify its condition. All Evotips were properly prepared as they display a pale-colored SPE material disc with visible solvent above it.\n5. Placed the verified Evotips into the prepared Evotip boxes at S1 from A1 to A12.\n6. Placed empty Evotips, called Blanks, at S3 from A1 to A12.\n7. ❌ **Omitted:** Did not document the precise position of each placed Evotip during the recorded procedure.\n\n\n## Results\n- Did not check for visible Buffer A solution in boxes (1 cm depth)\n- All non-blank Evotips showed pale-colored SPE material discs & clear solvent meniscus above each SPE disc of each Evotip\n- Evotips were placed at S1 from A1 to A6 and blanks placed at S3 from A1 to A6.\n",
        "error_dict": """[
  {"Step": 1, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 2, "Benchmark": "Error", "Class": "Error", "Skill": "SpatialOrientation"},
  {"Step": 3, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 4, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 5, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 6, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 7, "Benchmark": "Error", "Class": "Omitted", "Skill": "GeneralKnowledge"}
]""",
        "comments": "Step 2: Box is not properly seated in rack.",
    },
    {
        "eval_set_name": "ConnectingColumnSampleLine_docuWithoutStandbyANDtimsControl",
        "protocol": """The page titled ""Connecting IonOpticks column & sample line"" contains following information: - ## Abstract\nThis protocol explains how to connect an IonOpticks column inserted into a timsTOF Ultra with an sample line of an Evosep.\n\n\n## Materials\n\n### Equipment\n- IonOpticks Column\n- timsTOF Ultra Mass Spectrometer\n  - Equipped with UltraSource ion source\n  - Equipped with column oven for temperature control\n- Evosep One LC System\n  - with sample line\n- NanoViper Adapter (black)\n- Pliers\n\n\n## Procedure\n*Estimated timing: 3 minutes*\n\n1. Verfy that the instrument is in standing by mode. If not press the on/off button to switch from operate mode to standing by mode.\n\n### Connect column and sample line\n2. Verify that an IonOpticks column is already inserted into the UltraSource of the timsTOF Ultra.\n3. Attach a black NanoViper adapter to the LC sample line.\n4. Remove any access liquid at the top of the nanoViper of the sample line for instance by snipping it off.\n5. Hold the column fititng of the IonOpticks column with a pliers.\n6. Hand-tighten the NanoViper of the LC sample line with the column fitting (Figure 1).\n   CRITICAL STEP: Be careful not to overtighten the connection between the IonOpticks column and the timsTOF Ultra. Otherwise you can damage the column and the LC sample line.\n7. Remove the NanoViper adapter so that the oven can be tightly closed.\n8. Draw the oven closer to the UltraSource, secure it with the screw on the bottom of the oven (Figure 2).\n9. Lift and place the metal grounding screw at the column-sample line connection to establish proper ESI spray grounding with the column oven.\n10. Close the lid of the oven.\n11. Verify the temperature at 50°C for IonOpticks columns, as indicated by three illuminated LEDs on the column oven, respectively.\n\n### Switch timsTOF to operate and idle flow\n12. In timsControl, transition the instrument to the operation mode by using the on/off button.\n13. Navigate to Hystar and ensure that the idle flow is active. If not right-click on the Evosep logo, choosing 'preparation', and then selecting 'idle flow' and 'Run'.\n14. Return to timsControl. Check the MS signal. It should be around 2.5x10^7.\n\n\n## Expected Results\n- In timsControl, signal intensity should be above 10^7\n- Stable signal in in timsControl (in the windows for Mobilogram (timsView) and mass spectrum)\n\n\n## Figures\n### Figure 1: Connection of a sample line of a LC system with a separation column and emitter.**\n\n### Figure 2: UltraSource.**\n\n\n## References\n1. Skowronek, P., Wallmann, G., Wahle, M. et al. An accessible workflow for high-sensitivity proteomics using parallel accumulation-serial fragmentation (PASEF). Nat Protoc (2025). https://doi.org/10.1038/s41596-024-01104-w\n""",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/ConnectingColumnSampleLine_docuWithoutStandbyANDtimsControl.mp4""",
        "recording_type": "camera",
        "ground_truth_lab_notes": """# Connecting IonOpticks Column & sample line of Evosep\n\n## Aim\nConnecting an IonOpticks column inserted into a timsTOF Ultra with an sample line of an Evosep.\n\n\n## Materials\n\n### Equipment\n- IonOpticks Column\n- timsTOF Ultra Mass Spectrometer\n  - Equipped with UltraSource ion source\n  - Equipped with column oven for temperature control\n- Evosep One LC System\n  - with sample line\n- NanoViper Adapter (black)\n- Pliers\n\n\n## Procedure\n*Timing: 3 minutes*\n\n1. ❌ **Omitted:** Did not verify that the instrument was in standing by mode.\n\n### Connected column and sample line\n2. Verified that an IonOpticks column was already inserted into the UltraSource of the timsTOF Ultra.\n3. Attached a black NanoViper adapter to the LC sample line.\n4. Removed any excess liquid at the top of the nanoViper of the sample line by snipping it off.\n5. Held the column fitting of the IonOpticks column with pliers.\n6. Hand-tightened the NanoViper of the LC sample line with the column fitting.\n7. Removed the NanoViper adapter.\n8. ❌ **Omitted:** Did not draw the oven closer to the UltraSource or secure it with the screw on the bottom of the oven.\n9. Lifted and placed the metal grounding screw at the column-sample line connection to establish proper ESI spray grounding with the column oven.\n10. Closed the lid of the oven.\n10.1 ➕ **Added:** Connected the oven to the electrical power supply.\n11. Verified the temperature was at 50°C for IonOpticks columns, as indicated by three illuminated LEDs on the column oven.\n\n### Switched timsTOF to operate and idle flow\n12. ❌ **Omitted:** Skipped to transition the instrument to the operation mode by using the on/off button in timsControl.\n13. ❌ **Omitted:** Did not navigate to Hystar and ensure that the idle flow was active.\n14. ❌ **Omitted:** Did not return to timsControl to check the MS signal.\n\n\n## Results\nLast part of the protocol is not on video. Outcome is unclear.\n""",
        "error_dict": """[{'Step': 1.0, 'Benchmark': 'Error', 'Class': 'Omitted', 'Skill': 'Fast'}, {'Step': 2.0, 'Benchmark': 'No Error', 'Class': 'N/A', 'Skill': 'N/A'}, {'Step': 3.0, 'Benchmark': 'No Error', 'Class': 'N/A', 'Skill': 'N/A'}, {'Step': 4.0, 'Benchmark': 'No Error', 'Class': 'N/A', 'Skill': 'N/A'}, {'Step': 5.0, 'Benchmark': 'No Error', 'Class': 'N/A', 'Skill': 'N/A'}, {'Step': 6.0, 'Benchmark': 'No Error', 'Class': 'N/A', 'Skill': 'N/A'}, {'Step': 7.0, 'Benchmark': 'No Error', 'Class': 'N/A', 'Skill': 'N/A'}, {'Step': 8.0, 'Benchmark': 'Error', 'Class': 'Omitted', 'Skill': 'ProteomicsKnowledge'}, {'Step': 9.0, 'Benchmark': 'No Error', 'Class': 'N/A', 'Skill': 'N/A'}, {'Step': 10.0, 'Benchmark': 'No Error', 'Class': 'N/A', 'Skill': 'N/A'}, {'Step': 10.1, 'Benchmark': 'Error', 'Class': 'Addition', 'Skill': 'ProteomicsKnowledge'}, {'Step': 11.0, 'Benchmark': 'No Error', 'Class': 'N/A', 'Skill': 'N/A'}, {'Step': 12.0, 'Benchmark': 'Error', 'Class': 'Omitted', 'Skill': 'Fast'}, {'Step': 13.0, 'Benchmark': 'Error', 'Class': 'Omitted', 'Skill': 'ProteomicsKnowledge'}, {'Step': 14.0, 'Benchmark': 'Error', 'Class': 'Omitted', 'Skill': 'ProteomicsKnowledge'}]""",
        "comments": "Did not verify if in standby, Added connecting oven, not on video: Switched timsTOF to operate and idle flow",
    },
    {
        "eval_set_name": "ESIsourceToUltraSource_docuCorrect",
        "protocol": "# Change source: ESI source to UltraSource\n\n\n## Abstract\nThis protocol describes the procedure for switching from the ESI source to UltraSource.\n\n\n## Materials\n\n### Equipment\n- timsTOF Ultra Mass Spectrometer:\n  - Equipped with an ESI ion source\n  - UltraSource ion source ready to attach\n- IonOpticks Column\n- Evosep One LC System with sample line\n- NanoViper Adapter (black)\n- Pliers\n\n\n## Procedure\n*Estimated timing: less than 10 minute*\n\n### Switch timsTOF to standby\n1. In timsControl, verfy if the instrument is in standing by mode. If not press the on/off button to switch from operate mode to standy by mode  (Figure 1).\n2. Verify that the syringe is inactive in the tab 'Source' under 'Syringe Pump'. If not click on 'Stop'.\n3. In the 'source' and 'source type' sections of timsControl, choose 'CaptiveSpray' but do not activate it yet.\n\n### Remove ESI source\n4. At the instrument, disconnect the peak connector of the sample tubing (Figure 2).\n5. Disconnect the nebulizer N₂ line.\n6. Remove the ESI source\xa0housing by hinging it out like a regular door.\n7. Put on laboratory gloves.\n8. Remove the spray shield\n   ! CAUTION: The spray shield is hot.\n9. Remove the capillary cap.\n   ! CAUTION: The capillary cap is hot.\n10. Inspect the capillary position. If it appears to be partially pulled out, gently push it back into proper position without blocking the gas flow.\n\n### Mount UltraSource\n11. Hinge the UltraSource door in and close it (Figure 3).\n12. Slide the UltraSource housing onto the source door and secure it by flipping the handles located at the top right and bottom left by 180°.\n13. Connect the filter tubing to the source.\n\n### Connect column and sample line\n14. Verify that an IonOpticks column is already inserted into the UltraSource of the timsTOF Ultra.\n15. Verify if the LC sample line has a black NanoViper adapter attached. If not, locate an adapter and securely attach it to the sample line.\n16. Remove any access liquid at the top of the nanoViper of the sample line for instance by snipping it off.\n17. Hold the column fititng of the IonOpticks column with a pliers.\n18. Hand-tighten the NanoViper of the LC sample line with the column fitting (Figure 3).\n   CRITICAL STEP: Be careful not to overtighten the connection between the IonOpticks column and the timsTOF Ultra. Otherwise you can damage the column and the LC sample line.\n19. Draw the oven closer to the UltraSource, secure it with the screw on the bottom of the oven (Figure 2).\n20. Remove the NanoViper adapter so that the oven can be tightly closed.\n21. Lift and place the metal grounding screw at the column-sample line connection to establish proper ESI spray grounding with the column oven.\n22. Close the lid of the oven.\n23. Connect the oven to the electrical power supply.\n24. Set the temperature at 50°C for IonOpticks columns, as indicated by three illuminated LEDs on the column oven. Blincking light indicates that the oven is heating up.\n    CRITICAL STEP: Ensure that the IonOpticks column is not left connected to the LC for an extended period unless the mass spectrometer is in operate mode; otherwise, debris may accumulate on the emitter, lead to spitting.\n\n### Switch timsTOF to operate and idle flow\n25. In timsCOntrol, activate the CaptiveSpray function in timsControl (Figure 1)\n26. Transition the instrument to the operation mode by using the on/off button.\n27. Navigate to Hystar and ensure that the idle flow is active. If not right-click on the Evosep logo, choosing 'preparation', and then selecting 'idle flow' and 'Run'.\n28. Return to timsControl and check the MS signal. It should be around 2.5x10^7.\n29. Monitor the internal pressures in the timsTOF device. The funnel pressur is acceptable within a 9-11 mbar range. Aim for a 10 mbar funnel pressure. Adjust the 'funnel pressure' wheel if necessary by turning counter clock-wise to reduce the pressure.\n\n\n## Expected Results\n- In timsControl, signal intensity should be above 10^7\n- Stable signal in timsControl (in the windows for Mobilogram (timsView) and mass spectrum)\n\n\n## Figures\n**Figure 1: TimsControl settings before changing source.**\n\n**Figure 2: Apollo ESI source and gas flow pressure control wheels.**\n\n**Figure 3: UltraSource.**\n\n\n## References\n1. Skowronek, P., Wallmann, G., Wahle, M. et al. An accessible workflow for high-sensitivity proteomics using parallel accumulation–serial fragmentation (PASEF). Nat Protoc (2025). https://doi.org/10.1038/s41596-024-01104-w\n2. timsTOF user manual\n",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/ESIsourceToUltraSource_docuCorrect.MP4""",
        "recording_type": "camera",
        "ground_truth_lab_notes": """# Change source: ESI source to UltraSource\n\n\n## Aim\nSwitching from the ESI source to UltraSource.\n\n\n## Materials\n\n### Equipment\n- timsTOF Ultra Mass Spectrometer:\n  - Equipped with an ESI ion source\n  - UltraSource ion source ready to attach\n- IonOpticks Column\n- Evosep One LC System with sample line\n- NanoViper Adapter (black)\n- Pliers\n\n\n## Procedure\n*Timing: 5 minute*\n\n### Switch timsTOF to standby\n1. At timsControl, verified that the instrument was in standing by mode.\n2. Verified that the syringe was inactive in the tab 'Source' under 'Syringe Pump'.\n3. In the 'source' and 'source type' sections of timsControl, chose 'CaptiveSpray' but did not activate it yet.\n\n### Remove ESI source\n4. At the instrument, disconnected the peak connector of the sample tubing.\n5. Disconnected the nebulizer N₂ line.\n6. Removed the ESI source\xa0housing by hinging it out like a regular door.\n7. Person wore already gloves from step 1.\n8. Removed the spray shield, and\n9. Removed the capillary cap.\n10. Inspected the capillary position. If it appeared to be partially pulled out, gently pushed it back into proper position without blocking the gas flow.\n\n### Mount UltraSource\n11. Hinged the UltraSource door in and closed it.\n12. Slid the UltraSource housing onto the source door and secured it by flipping the handles located at the top right and bottom left by 180°.\n13. Connected the filter tubing to the source.\n\n### Connect column and sample line\n14. Verified that the IonOpticks column was already inserted into the UltraSource of the timsTOF Ultra.\n15. Attached a black NanoViper adapter to the sample line.\n16. Removed any access liquid at the top of the nanoViper of the sample line by snipping it off.\n17. Held the column fitting of the IonOpticks column with pliers.\n18. Hand-tightened the NanoViper of the LC sample line with the column fitting.\n19. Drew the oven closer to the UltraSource, secured it with the screw on the bottom of the oven.\n20. Removed the NanoViper adapter so that the oven could be tightly closed.\n21. Lifted and placed the metal grounding screw at the column-sample line connection to establish proper ESI spray grounding with the column oven.\n22. Closed the lid of the oven.\n23. Connected the oven to the electrical power supply.\n24. Set the temperature at 50°C for IonOpticks columns, as indicated by three illuminated LEDs on the column oven. Blinking light indicated that the oven was heating up.\n\n### Switch timsTOF to operate and idle flow\n25. In timsControl, activated the CaptiveSpray function.\n26. Transitioned the instrument to the operation mode by using the on/off button.\n27. ❌ **Omitted:** The researcher did not navigate to Hystar and ensured that the idle flow was active. If not right-clicked on the Evosep logo, choosing 'preparation', and then selecting 'idle flow' and 'Run'.\n28. Returned to timsControl and checked the MS signal. It was around 2.5x10^7.\n29. Monitored the internal pressures in the timsTOF device. Adjusted the 'funnel pressure' wheel aiming for a 10 mbar funnel pressure.\n\n\n## Results\n- In timsControl, signal intensity was above 10^7\n- Stable signal in timsControl (in the windows for Mobilogram (timsView) and mass spectrum)\n""",
        "error_dict": """[
  {"Step": 1, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 2, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 3, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 4, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 5, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 6, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 7, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 8, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 9, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 10, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 11, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 12, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 13, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 14, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 15, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 16, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 17, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 18, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 19, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 20, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 21, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 22, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 23, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 24, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 25, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 26, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 27, "Benchmark": "Error", "Class": "Omitted", "Skill": "ProteomicsKnowledge"},
  {"Step": 28, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 29, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"}
]""",
        "comments": "",
    },
    {
        "eval_set_name": "ESIsourceToUltraSource_docuFogotOvenPowerSupply",
        "protocol": "# Change source: ESI source to UltraSource\n\n\n## Abstract\nThis protocol describes the procedure for switching from the ESI source to UltraSource.\n\n\n## Materials\n\n### Equipment\n- timsTOF Ultra Mass Spectrometer:\n  - Equipped with an ESI ion source\n  - UltraSource ion source ready to attach\n- IonOpticks Column\n- Evosep One LC System with sample line\n- NanoViper Adapter (black)\n- Pliers\n\n\n## Procedure\n*Estimated timing: less than 10 minute*\n\n### Switch timsTOF to standby\n1. In timsControl, verfy if the instrument is in standing by mode. If not press the on/off button to switch from operate mode to standy by mode  (Figure 1).\n2. Verify that the syringe is inactive in the tab 'Source' under 'Syringe Pump'. If not click on 'Stop'.\n3. In the 'source' and 'source type' sections of timsControl, choose 'CaptiveSpray' but do not activate it yet.\n\n### Remove ESI source\n4. At the instrument, disconnect the peak connector of the sample tubing (Figure 2).\n5. Disconnect the nebulizer N₂ line.\n6. Remove the ESI source\xa0housing by hinging it out like a regular door.\n7. Put on laboratory gloves.\n8. Remove the spray shield\n   ! CAUTION: The spray shield is hot.\n9. Remove the capillary cap.\n   ! CAUTION: The capillary cap is hot.\n10. Inspect the capillary position. If it appears to be partially pulled out, gently push it back into proper position without blocking the gas flow.\n\n### Mount UltraSource\n11. Hinge the UltraSource door in and close it (Figure 3).\n12. Slide the UltraSource housing onto the source door and secure it by flipping the handles located at the top right and bottom left by 180°.\n13. Connect the filter tubing to the source.\n\n### Connect column and sample line\n14. Verify that an IonOpticks column is already inserted into the UltraSource of the timsTOF Ultra.\n15. Verify if the LC sample line has a black NanoViper adapter attached. If not, locate an adapter and securely attach it to the sample line.\n16. Remove any access liquid at the top of the nanoViper of the sample line for instance by snipping it off.\n17. Hold the column fititng of the IonOpticks column with a pliers.\n18. Hand-tighten the NanoViper of the LC sample line with the column fitting (Figure 3).\n   CRITICAL STEP: Be careful not to overtighten the connection between the IonOpticks column and the timsTOF Ultra. Otherwise you can damage the column and the LC sample line.\n19. Draw the oven closer to the UltraSource, secure it with the screw on the bottom of the oven (Figure 2).\n20. Remove the NanoViper adapter so that the oven can be tightly closed.\n21. Lift and place the metal grounding screw at the column-sample line connection to establish proper ESI spray grounding with the column oven.\n22. Close the lid of the oven.\n23. Connect the oven to the electrical power supply.\n24. Set the temperature at 50°C for IonOpticks columns, as indicated by three illuminated LEDs on the column oven. Blincking light indicates that the oven is heating up.\n    CRITICAL STEP: Ensure that the IonOpticks column is not left connected to the LC for an extended period unless the mass spectrometer is in operate mode; otherwise, debris may accumulate on the emitter, lead to spitting.\n\n### Switch timsTOF to operate and idle flow\n25. In timsCOntrol, activate the CaptiveSpray function in timsControl (Figure 1)\n26. Transition the instrument to the operation mode by using the on/off button.\n27. Navigate to Hystar and ensure that the idle flow is active. If not right-click on the Evosep logo, choosing 'preparation', and then selecting 'idle flow' and 'Run'.\n28. Return to timsControl and check the MS signal. It should be around 2.5x10^7.\n29. Monitor the internal pressures in the timsTOF device. The funnel pressur is acceptable within a 9-11 mbar range. Aim for a 10 mbar funnel pressure. Adjust the 'funnel pressure' wheel if necessary by turning counter clock-wise to reduce the pressure.\n\n\n## Expected Results\n- In timsControl, signal intensity should be above 10^7\n- Stable signal in timsControl (in the windows for Mobilogram (timsView) and mass spectrum)\n\n\n## Figures\n**Figure 1: TimsControl settings before changing source.**\n\n**Figure 2: Apollo ESI source and gas flow pressure control wheels.**\n\n**Figure 3: UltraSource.**\n\n\n## References\n1. Skowronek, P., Wallmann, G., Wahle, M. et al. An accessible workflow for high-sensitivity proteomics using parallel accumulation–serial fragmentation (PASEF). Nat Protoc (2025). https://doi.org/10.1038/s41596-024-01104-w\n2. timsTOF user manual\n",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/ESIsourceToUltraSource_docuFogotOvenPowerSupply.MP4""",
        "recording_type": "camera",
        "ground_truth_lab_notes": """# Change source: ESI source to UltraSource\n\n\n## Aim\nSwitching from the ESI source to UltraSource.\n\n\n## Materials\n\n### Equipment\n- timsTOF Ultra Mass Spectrometer:\n  - Equipped with an ESI ion source\n  - UltraSource ion source ready to attach\n- IonOpticks Column\n- Evosep One LC System with sample line\n- NanoViper Adapter (black)\n- Pliers\n\n\n## Procedure\n*Timing: 5 minute*\n\nPrerequisite 1: Person wore gloves.\n\n### Switch timsTOF to standby\n1. In timsControl, verified that the instrument was in standing by mode.\n2. Verified that the syringe was inactive in the tab 'Source' under 'Syringe Pump'.\n3. In the 'source' and 'source type' sections of timsControl, chose 'CaptiveSpray' but did not activate it yet.\n\n### Remove ESI source\n4. At the instrument, disconnected the peak connector of the sample tubing.\n5. Disconnected the nebulizer N₂ line.\n6. Removed the ESI source\xa0housing by hinging it out like a regular door.\n7. ❌ **Omitted:** Did not put on laboratory gloves.\n8. Removed the spray shield.\n9. Removed the capillary cap.\n10. ❌ **Omitted:** Did not inspected the capillary position.\n\n### Mount UltraSource\n11. Hinged the UltraSource door in and closed it.\n12. Slid the UltraSource housing onto the source door and secured it by flipping the handles located at the top right and bottom left by 180°.\n13. Connected the filter tubing to the source.\n\n### Connect column and sample line\n14. Verified that the IonOpticks column was already inserted into the UltraSource of the timsTOF Ultra.\n15. Attached a black NanoViper adapter to the sample line.\n16. Removed any access liquid at the top of the nanoViper of the sample line by snipping it off.\n17. Held the column fitting of the IonOpticks column with pliers.\n18. Hand-tightened the NanoViper of the LC sample line with the column fitting.\n20. ⚠️ **Deviation: Altered step order** Removed the NanoViper adapter so that the oven could be tightly closed.\n19. Drew the oven closer to the UltraSource, secured it with the screw on the bottom of the oven.\n21. Lifted and placed the metal grounding screw at the column-sample line connection to establish proper ESI spray grounding with the column oven.\n22. Closed the lid of the oven.\n23. ❌ **Omitted:** Connected the oven to the electrical power supply.\n24. ❌ **Omitted:** Set the temperature at 50°C for IonOpticks columns, as indicated by three illuminated LEDs on the column oven. Blinking light indicated that the oven was heating up.\n\n### Switch timsTOF to operate and idle flow\n25. In timsControl, activated the CaptiveSpray function.\n29. ⚠️ **Deviation: Altered step order** Monitored the internal pressures in the timsTOF device. Adjusted the 'funnel pressure' wheel aiming for a 10 mbar funnel pressure.\n26. Transitioned the instrument to the operation mode by using the on/off button.\n27. ❌ **Omitted:** The researcher did not navigate to Hystar and ensured that the idle flow was active. If not right-clicked on the Evosep logo, choosing 'preparation', and then selecting 'idle flow' and 'Run'.\n28. Still in timsControl, checked the MS signal. It was around 2.5x10^7.\n\n\n## Results\n- In timsControl, signal intensity was above 10^7\n- Stable signal in timsControl (in the windows for Mobilogram (timsView) and mass spectrum)\n""",
        "error_dict": """[
{"Step": 1, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 2, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 3, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 4, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 5, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 6, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 7, "Benchmark": "Error", "Class": "Omitted", "Skill": "GeneralKnowledge"},
{"Step": 8, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 9, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 10, "Benchmark": "Error", "Class": "Omitted", "Skill": "ProteomicsKnowledge"},
{"Step": 11, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 12, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 13, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 14, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 15, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 16, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 17, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 18, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 19, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 20, "Benchmark": "Error", "Class": "Deviation", "Skill": "ProteomicsKnowledge"},
{"Step": 21, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 22, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 23, "Benchmark": "Error", "Class": "Omitted", "Skill": "ProteomicsKnowledge"},
{"Step": 24, "Benchmark": "Error", "Class": "Omitted", "Skill": "ProteomicsKnowledge"},
{"Step": 25, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 26, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 27, "Benchmark": "Error", "Class": "Omitted", "Skill": "ProteomicsKnowledge"},
{"Step": 28, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 29, "Benchmark": "Error", "Class": "Deviation", "Skill": "ProteomicsKnowledge"}
]""",
        "comments": "forgot to connect the oven, some steps omitted or order of steps changed",
    },
    {
        "eval_set_name": "UltraSourceToESIsource_docuCorrect",
        "protocol": """# Change source: UltraSource to ESI source\n\n\n## Abstract\nThis protocol describes the procedure for switching from the UltraSource to the ESI source.\n\n\n## Materials\n\n### Equipment\n- timsTOF Ultra Mass Spectrometer:\n  - Equipped with an UltraSource ion source\n  - ESI source on the side to attach\n\n\n## Procedure\n*Estimated timing: less than 10 minute*\n\n### Switch TimsControl to 'Standby' mode\n1. Verify that the IonOpticks column from an Evosep System is disconnected. If not disconnect them.\n   Critical step: This could be an opportune moment to perform the weekly LC system maintenance.\n2. In timsControl, verfy that the software is in standby mode\n3. Navigate to the 'Source', then proceed to 'SourceType' and select 'ESI'. However, do not activate the source at this moment (Figure 1).\n\n### Remove UltraSource\n4. Disconnect the power supply of the oven.\n5. Disconnect the filter tubing.\n6. Rotate the top right and bottom left handles of the UltraSource housing by 180 degrees.\n7. Slide off the UltraSource housing from both the source door and the glass capillary.\n8. Set the UltraSource housing on the bench.\n   Critical step: Never attempt to open the source door while the UltraSource is connected with the glass capillary (Figure 2). Such an action will cause breakage of the glass capillary.\n9. Remove the source door by opening it and unhinging it, similar to the motion of a normal door.\n10. Set it aside (Figure 2).\n\n### Mount Apollo ESI source\n11. Put on gloves.\n12. Attach the capillary cap securely to the glass capillary (Figure 3)\n   Critical step: Try not to block the vacuum flow to avoid contamination of the timsTOF for instance by blocking the whole of the capillary cap.\n13. Attach the spray shield to the desolvation stage housing.\n14. Hinging the ESI source\xa0housing into position as it would be a door and close it.\n15. Connect the sample inlet of the ESI source and the peak tubing lines, which originate from the syringe, by turning it clock-wise.\n16. Connect the nebulizer gas inlet to the N₂ line.\n\n### Prepare the setup by loading the syringe with Tuning Mix\n17. Remove old solvent.\n18. Withdraw new Tuning Mix liquid.\n19. Ensure it is devoid of air bubbles.\n20. Connect the syringe to the peak tubing of the sample line.\n21. Mount this syringe within the external syringe pump setup. The golden button allows to move the syringe holder to arrange the syringe as in Figure 3.\n22. Press some solved out of the syringe to fill the sample line.\n\n### Switch TimsControl to 'Operate' mode\n23. Activate the ESI source within the TimsControl software (Figure 1).\n24. Transition the instrument into 'operate' mode by clicking on the on/off symbol.\n25. Begin flow at the syringe, ensuring that the following settings are in place in the tab 'Source' (Figure 1): Syringe: Hamilton 500 µL; Flow Rate: 3 µL/min. Press start. Keep the button next to start/stop pressed until a signal is observed in the 'Chromatogram View' and 'TIMS View' windows, which usually happens within 30 seconds.\n\n## Expected Results\n- Signal intensity should reach approximately 1.5x10^7\n- Stable signal in in timsControl (in the windows for Mobilogram (timsView) and mass spectrum)\n\n## Figures\n### Figure 1: TimsControl settings before changing source\n\n### Figure 2: UltraSource.\n\n### Figure 3: Apollo ESI source and gas flow pressure control wheels.\n\n## References\n1. Skowronek, P., Wallmann, G., Wahle, M. et al. An accessible workflow for high-sensitivity proteomics using parallel accumulation–serial fragmentation (PASEF). Nat Protoc (2025). https://doi.org/10.1038/s41596-024-01104-w\n2. TODO: timsTOF user manual\n""",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/UltraSourceToESIsource_docuCorrect.MP4""",
        "recording_type": "camera",
        "ground_truth_lab_notes": """# Change source: UltraSource to ESI source\n\n\n## Aim\nSwitching from the UltraSource to the ESI source.\n\n\n## Materials\n\n### Equipment\n- timsTOF Ultra Mass Spectrometer:\n  - Equipped with an UltraSource ion source\n  - ESI source on the side to attach\n\n\n## Procedure\n*Timing: less than 10 minute*\n\n### Switch TimsControl to 'Standby' mode\n1. Verified that the IonOpticks column from an Evosep System was disconnected.\n2. In timsControl, verified that the software was in standby mode.\n3. Navigated to the 'Source', then proceeded to 'SourceType' and selected 'ESI'. Did not activate the source at this moment.\n\n### Remove UltraSource\n4. Disconnected the power supply of the oven.\n5. Disconnected the filter tubing.\n6. Rotated the top right and bottom left handles of the UltraSource housing by 180 degrees.\n7. Slid off the UltraSource housing from both the source door and the glass capillary.\n8. Set the UltraSource housing on the bench.\n9. Removed the source door by opening it and unhinging it, similar to the motion of a normal door.\n10. Set it aside.\n\n### Mount Apollo ESI source\n11. Put on gloves.\n12. Attached the capillary cap securely to the glass capillary.\n13. Attached the spray shield to the desolvation stage housing.\n14. Hinged the ESI source\xa0housing into position as if it were a door and closed it.\n15. Connected the sample inlet of the ESI source and the peak tubing lines, which originated from the syringe, by turning it clockwise.\n16. Connected the nebulizer gas inlet to the N₂ line.\n\n### Prepare the setup by loading the syringe with Tuning Mix\n17. Removed old solvent.\n18. Withdrew new Tuning Mix liquid.\n19. Ensured it was devoid of air bubbles.\n20. Connected the syringe to the peak tubing of the sample line.\n21. Mounted this syringe within the external syringe pump setup. The golden button allowed movement of the syringe holder to arrange the syringe as in Figure 3.\n22. Pressed some solvent out of the syringe to fill the sample line.\n\n### Switch TimsControl to 'Operate' mode\n23. Activated the ESI source within the TimsControl software.\n24. Transitioned the instrument into 'operate' mode by clicking on the on/off symbol.\n25. Began flow at the syringe, ensuring that the following settings were in place in the tab 'Source': Syringe: Hamilton 500 µL; Flow Rate: 3 µL/min. Pressed start. Kept the button next to start/stop pressed until a signal was observed in the 'Chromatogram View' and 'TIMS View' windows, which happened within 30 seconds.\n\n\n## Results\n- Signal intensity reached approximately 1.5x10^7\n- Stable signal was observed in timsControl (in the windows for Mobilogram (timsView) and mass spectrum)\n""",
        "error_dict": """[
  {"Step": 1, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 2, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 3, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 4, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 5, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 6, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 7, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 8, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 9, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 10, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 11, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 12, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 13, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 14, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 15, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 16, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 17, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 18, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 19, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 20, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 21, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 22, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 23, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 24, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 25, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"}
]""",
        "comments": "",
    },
    {
        "eval_set_name": "UltraSourceToESIsource_docuForgotN2Line",
        "protocol": """# Change source: UltraSource to ESI source\n\n\n## Abstract\nThis protocol describes the procedure for switching from the UltraSource to the ESI source.\n\n\n## Materials\n\n### Equipment\n- timsTOF Ultra Mass Spectrometer:\n  - Equipped with an UltraSource ion source\n  - ESI source on the side to attach\n\n\n## Procedure\n*Estimated timing: less than 10 minute*\n\n### Switch TimsControl to 'Standby' mode\n1. Verify that the IonOpticks column from an Evosep System is disconnected. If not disconnect them.\n   Critical step: This could be an opportune moment to perform the weekly LC system maintenance.\n2. In timsControl, verfy that the software is in standby mode\n3. Navigate to the 'Source', then proceed to 'SourceType' and select 'ESI'. However, do not activate the source at this moment (Figure 1).\n\n### Remove UltraSource\n4. Disconnect the power supply of the oven.\n5. Disconnect the filter tubing.\n6. Rotate the top right and bottom left handles of the UltraSource housing by 180 degrees.\n7. Slide off the UltraSource housing from both the source door and the glass capillary.\n8. Set the UltraSource housing on the bench.\n   Critical step: Never attempt to open the source door while the UltraSource is connected with the glass capillary (Figure 2). Such an action will cause breakage of the glass capillary.\n9. Remove the source door by opening it and unhinging it, similar to the motion of a normal door.\n10. Set it aside (Figure 2).\n\n### Mount Apollo ESI source\n11. Put on gloves.\n12. Attach the capillary cap securely to the glass capillary (Figure 3)\n   Critical step: Try not to block the vacuum flow to avoid contamination of the timsTOF for instance by blocking the whole of the capillary cap.\n13. Attach the spray shield to the desolvation stage housing.\n14. Hinging the ESI source\xa0housing into position as it would be a door and close it.\n15. Connect the sample inlet of the ESI source and the peak tubing lines, which originate from the syringe, by turning it clock-wise.\n16. Connect the nebulizer gas inlet to the N₂ line.\n\n### Prepare the setup by loading the syringe with Tuning Mix\n17. Remove old solvent.\n18. Withdraw new Tuning Mix liquid.\n19. Ensure it is devoid of air bubbles.\n20. Connect the syringe to the peak tubing of the sample line.\n21. Mount this syringe within the external syringe pump setup. The golden button allows to move the syringe holder to arrange the syringe as in Figure 3.\n22. Press some solved out of the syringe to fill the sample line.\n\n### Switch TimsControl to 'Operate' mode\n23. Activate the ESI source within the TimsControl software (Figure 1).\n24. Transition the instrument into 'operate' mode by clicking on the on/off symbol.\n25. Begin flow at the syringe, ensuring that the following settings are in place in the tab 'Source' (Figure 1): Syringe: Hamilton 500 µL; Flow Rate: 3 µL/min. Press start. Keep the button next to start/stop pressed until a signal is observed in the 'Chromatogram View' and 'TIMS View' windows, which usually happens within 30 seconds.\n\n## Expected Results\n- Signal intensity should reach approximately 1.5x10^7\n- Stable signal in in timsControl (in the windows for Mobilogram (timsView) and mass spectrum)\n\n## Figures\n### Figure 1: TimsControl settings before changing source\n\n### Figure 2: UltraSource.\n\n### Figure 3: Apollo ESI source and gas flow pressure control wheels.\n\n## References\n1. Skowronek, P., Wallmann, G., Wahle, M. et al. An accessible workflow for high-sensitivity proteomics using parallel accumulation–serial fragmentation (PASEF). Nat Protoc (2025). https://doi.org/10.1038/s41596-024-01104-w\n2. TODO: timsTOF user manual\n""",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/UltraSourceToESIsource_docuForgotN2Line.MP4""",
        "recording_type": "camera",
        "ground_truth_lab_notes": """# Change source: UltraSource to ESI source\n\n\n## Aim\nSwitching from the UltraSource to the ESI source.\n\n\n## Materials\n\n### Equipment\n- timsTOF Ultra Mass Spectrometer:\n  - Equipped with an UltraSource ion source\n  - ESI source on the side to attach\n\n\n## Procedure\n*Timing: less than 10 minute*\n\n### Switch TimsControl to 'Standby' mode\n1. Verified that the IonOpticks column from an Evosep System was disconnected.\n2. In timsControl, verified that the software was in standby mode.\n3. Navigated to the 'Source', then proceeded to 'SourceType' and selected 'ESI'. Did not activate the source at this moment.\n\n### Remove UltraSource\n4. Disconnected the power supply of the oven.\n5. Disconnected the filter tubing.\n6. Rotated the top right and bottom left handles of the UltraSource housing by 180 degrees.\n7. Slid off the UltraSource housing from both the source door and the glass capillary.\n8. Set the UltraSource housing on the bench.\n9. Removed the source door by opening it and unhinging it, similar to the motion of a normal door.\n10. Set it aside.\n\n### Mount Apollo ESI source\n11. Put on gloves.\n12. Attached the capillary cap securely to the glass capillary.\n13. Attached the spray shield to the desolvation stage housing.\n14. Hinged the ESI source\xa0housing into position as if it were a door and closed it.\n15. Connected the sample inlet of the ESI source and the peak tubing lines, which originated from the syringe, by turning it clockwise.\n16. ❌ **Omitted:** Did not connect the nebulizer gas inlet to the N₂ line.\n\n### Prepare the setup by loading the syringe with Tuning Mix\n17. Removed old solvent.\n18. Withdrew new Tuning Mix liquid.\n19. Ensured it was devoid of air bubbles.\n20. Connected the syringe to the peak tubing of the sample line.\n21. Mounted this syringe within the external syringe pump setup. The golden button allowed movement of the syringe holder to arrange the syringe as in Figure 3.\n22. Pressed some solvent out of the syringe to fill the sample line.\n\n### Switch TimsControl to 'Operate' mode\n23. Activated the ESI source within the TimsControl software.\n24. Transitioned the instrument into 'operate' mode by clicking on the on/off symbol.\n25. Began flow at the syringe, ensuring that the following settings were in place in the tab 'Source': Syringe: Hamilton 500 µL; Flow Rate: 3 µL/min. Pressed start. Kept the button next to start/stop pressed until a signal was observed in the 'Chromatogram View' and 'TIMS View' windows, which happened within 30 seconds.\n\n\n## Results\n- Lower signal than usually since the nebulizer gas inlet was not connected to the N₂ line.\n""",
        "error_dict": """[
{"Step": 1, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 2, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 3, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 4, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 5, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 6, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 7, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 8, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 9, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 10, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 11, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 12, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 13, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 14, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 15, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 16, "Benchmark": "Error", "Class": "Omitted", "Skill": "ProteomicsKnowledge"},
{"Step": 17, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 18, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 19, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 20, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 21, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 22, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 23, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 24, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 25, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"}
]""",
        "comments": "Forgot N2 line",
    },
    {
        "eval_set_name": "UltraSourceToESIsource_docuForgotGlovesANDCapillaryCap",
        "protocol": """# Change source: UltraSource to ESI source\n\n\n## Abstract\nThis protocol describes the procedure for switching from the UltraSource to the ESI source.\n\n\n## Materials\n\n### Equipment\n- timsTOF Ultra Mass Spectrometer:\n  - Equipped with an UltraSource ion source\n  - ESI source on the side to attach\n\n\n## Procedure\n*Estimated timing: less than 10 minute*\n\n### Switch TimsControl to 'Standby' mode\n1. Verify that the IonOpticks column from an Evosep System is disconnected. If not disconnect them.\n   Critical step: This could be an opportune moment to perform the weekly LC system maintenance.\n2. In timsControl, verfy that the software is in standby mode\n3. Navigate to the 'Source', then proceed to 'SourceType' and select 'ESI'. However, do not activate the source at this moment (Figure 1).\n\n### Remove UltraSource\n4. Disconnect the power supply of the oven.\n5. Disconnect the filter tubing.\n6. Rotate the top right and bottom left handles of the UltraSource housing by 180 degrees.\n7. Slide off the UltraSource housing from both the source door and the glass capillary.\n8. Set the UltraSource housing on the bench.\n   Critical step: Never attempt to open the source door while the UltraSource is connected with the glass capillary (Figure 2). Such an action will cause breakage of the glass capillary.\n9. Remove the source door by opening it and unhinging it, similar to the motion of a normal door.\n10. Set it aside (Figure 2).\n\n### Mount Apollo ESI source\n11. Put on gloves.\n12. Attach the capillary cap securely to the glass capillary (Figure 3)\n   Critical step: Try not to block the vacuum flow to avoid contamination of the timsTOF for instance by blocking the whole of the capillary cap.\n13. Attach the spray shield to the desolvation stage housing.\n14. Hinging the ESI source\xa0housing into position as it would be a door and close it.\n15. Connect the sample inlet of the ESI source and the peak tubing lines, which originate from the syringe, by turning it clock-wise.\n16. Connect the nebulizer gas inlet to the N₂ line.\n\n### Prepare the setup by loading the syringe with Tuning Mix\n17. Remove old solvent.\n18. Withdraw new Tuning Mix liquid.\n19. Ensure it is devoid of air bubbles.\n20. Connect the syringe to the peak tubing of the sample line.\n21. Mount this syringe within the external syringe pump setup. The golden button allows to move the syringe holder to arrange the syringe as in Figure 3.\n22. Press some solved out of the syringe to fill the sample line.\n\n### Switch TimsControl to 'Operate' mode\n23. Activate the ESI source within the TimsControl software (Figure 1).\n24. Transition the instrument into 'operate' mode by clicking on the on/off symbol.\n25. Begin flow at the syringe, ensuring that the following settings are in place in the tab 'Source' (Figure 1): Syringe: Hamilton 500 µL; Flow Rate: 3 µL/min. Press start. Keep the button next to start/stop pressed until a signal is observed in the 'Chromatogram View' and 'TIMS View' windows, which usually happens within 30 seconds.\n\n## Expected Results\n- Signal intensity should reach approximately 1.5x10^7\n- Stable signal in in timsControl (in the windows for Mobilogram (timsView) and mass spectrum)\n\n## Figures\n### Figure 1: TimsControl settings before changing source\n\n### Figure 2: UltraSource.\n\n### Figure 3: Apollo ESI source and gas flow pressure control wheels.\n\n## References\n1. Skowronek, P., Wallmann, G., Wahle, M. et al. An accessible workflow for high-sensitivity proteomics using parallel accumulation–serial fragmentation (PASEF). Nat Protoc (2025). https://doi.org/10.1038/s41596-024-01104-w\n2. TODO: timsTOF user manual\n""",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/UltraSourceToESIsource_docuForgotGlovesANDCapillaryCap.MP4""",
        "recording_type": "camera",
        "ground_truth_lab_notes": """# Change source: UltraSource to ESI source\n\n\n## Aim\nSwitching from the UltraSource to the ESI source.\n\n\n## Materials\n\n### Equipment\n- timsTOF Ultra Mass Spectrometer:\n  - Equipped with an UltraSource ion source\n  - ESI source on the side to attach\n\n\n## Procedure\n*Timing: less than 10 minute*\n\n### Switch TimsControl to 'Standby' mode\n1. Verified that the IonOpticks column from an Evosep System was disconnected.\n2. In timsControl, verified that the software was in standby mode.\n3. Navigated to the 'Source', then proceeded to 'SourceType' and selected 'ESI'. Did not activate the source at this moment.\n\n### Remove UltraSource\n4. Disconnected the power supply of the oven.\n5. Disconnected the filter tubing.\n6. Rotated the top right and bottom left handles of the UltraSource housing by 180 degrees.\n7. Slid off the UltraSource housing from both the source door and the glass capillary.\n8. Set the UltraSource housing on the bench.\n9. Removed the source door by opening it and unhinging it, similar to the motion of a normal door.\n10. Set it aside.\n\n### Mount Apollo ESI source\n11. ❌ **Omitted:** Did not put on gloves.\n12. ❌ **Omitted:** Did not attach the capillary cap securely to the glass capillary.\n13. Attached the spray shield to the desolvation stage housing.\n14. Hinged the ESI source\xa0housing into position as if it were a door and closed it.\n15. Connected the sample inlet of the ESI source and the peak tubing lines, which originated from the syringe, by turning it clockwise.\n16. Connected the nebulizer gas inlet to the N₂ line.\n\n### Prepare the setup by loading the syringe with Tuning Mix\n17. Removed old solvent.\n18. Withdrew new Tuning Mix liquid.\n19. Ensured it was devoid of air bubbles.\n20. Connected the syringe to the peak tubing of the sample line.\n21. Mounted this syringe within the external syringe pump setup. The golden button allowed movement of the syringe holder to arrange the syringe as in Figure 3.\n22. Pressed some solvent out of the syringe to fill the sample line.\n\n### Switch TimsControl to 'Operate' mode\n23. Activated the ESI source within the TimsControl software.\n24. Transitioned the instrument into 'operate' mode by clicking on the on/off symbol.\n25. Began flow at the syringe, ensuring that the following settings were in place in the tab 'Source': Syringe: Hamilton 500 µL; Flow Rate: 3 µL/min. Pressed start. Kept the button next to start/stop pressed until a signal was observed in the 'Chromatogram View' and 'TIMS View' windows, which happened within 30 seconds.\n\n\n\n## Results\n- Signal intensity reached approximately 1.5x10^7\n- Stable signal was observed in timsControl (in the windows for Mobilogram (timsView) and mass spectrum)\n""",
        "error_dict": """[
{"Step": 1, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 2, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 3, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 4, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 5, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 6, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 7, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 8, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 9, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 10, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 11, "Benchmark": "Error", "Class": "Omitted", "Skill": "GeneralKnowledge"},
{"Step": 12, "Benchmark": "Error", "Class": "Omitted", "Skill": "ProteomicsKnowledge"},
{"Step": 13, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 14, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 15, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 16, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 17, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 18, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 19, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 20, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 21, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 22, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 23, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 24, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 25, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"}
]""",
        "comments": "Forgot gloves and capillary cap",
    },
    {
        "eval_set_name": "UltraSourceToESIsource_docuForgotCapillaryCap",
        "protocol": """# Change source: UltraSource to ESI source\n\n\n## Abstract\nThis protocol describes the procedure for switching from the UltraSource to the ESI source.\n\n\n## Materials\n\n### Equipment\n- timsTOF Ultra Mass Spectrometer:\n  - Equipped with an UltraSource ion source\n  - ESI source on the side to attach\n\n\n## Procedure\n*Estimated timing: less than 10 minute*\n\n### Switch TimsControl to 'Standby' mode\n1. Verify that the IonOpticks column from an Evosep System is disconnected. If not disconnect them.\n   Critical step: This could be an opportune moment to perform the weekly LC system maintenance.\n2. In timsControl, verfy that the software is in standby mode\n3. Navigate to the 'Source', then proceed to 'SourceType' and select 'ESI'. However, do not activate the source at this moment (Figure 1).\n\n### Remove UltraSource\n4. Disconnect the power supply of the oven.\n5. Disconnect the filter tubing.\n6. Rotate the top right and bottom left handles of the UltraSource housing by 180 degrees.\n7. Slide off the UltraSource housing from both the source door and the glass capillary.\n8. Set the UltraSource housing on the bench.\n   Critical step: Never attempt to open the source door while the UltraSource is connected with the glass capillary (Figure 2). Such an action will cause breakage of the glass capillary.\n9. Remove the source door by opening it and unhinging it, similar to the motion of a normal door.\n10. Set it aside (Figure 2).\n\n### Mount Apollo ESI source\n11. Put on gloves.\n12. Attach the capillary cap securely to the glass capillary (Figure 3)\n   Critical step: Try not to block the vacuum flow to avoid contamination of the timsTOF for instance by blocking the whole of the capillary cap.\n13. Attach the spray shield to the desolvation stage housing.\n14. Hinging the ESI source\xa0housing into position as it would be a door and close it.\n15. Connect the sample inlet of the ESI source and the peak tubing lines, which originate from the syringe, by turning it clock-wise.\n16. Connect the nebulizer gas inlet to the N₂ line.\n\n### Prepare the setup by loading the syringe with Tuning Mix\n17. Remove old solvent.\n18. Withdraw new Tuning Mix liquid.\n19. Ensure it is devoid of air bubbles.\n20. Connect the syringe to the peak tubing of the sample line.\n21. Mount this syringe within the external syringe pump setup. The golden button allows to move the syringe holder to arrange the syringe as in Figure 3.\n22. Press some solved out of the syringe to fill the sample line.\n\n### Switch TimsControl to 'Operate' mode\n23. Activate the ESI source within the TimsControl software (Figure 1).\n24. Transition the instrument into 'operate' mode by clicking on the on/off symbol.\n25. Begin flow at the syringe, ensuring that the following settings are in place in the tab 'Source' (Figure 1): Syringe: Hamilton 500 µL; Flow Rate: 3 µL/min. Press start. Keep the button next to start/stop pressed until a signal is observed in the 'Chromatogram View' and 'TIMS View' windows, which usually happens within 30 seconds.\n\n## Expected Results\n- Signal intensity should reach approximately 1.5x10^7\n- Stable signal in in timsControl (in the windows for Mobilogram (timsView) and mass spectrum)\n\n## Figures\n### Figure 1: TimsControl settings before changing source\n\n### Figure 2: UltraSource.\n\n### Figure 3: Apollo ESI source and gas flow pressure control wheels.\n\n## References\n1. Skowronek, P., Wallmann, G., Wahle, M. et al. An accessible workflow for high-sensitivity proteomics using parallel accumulation–serial fragmentation (PASEF). Nat Protoc (2025). https://doi.org/10.1038/s41596-024-01104-w\n2. TODO: timsTOF user manual\n""",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/UltraSourceToESIsource_docuForgotCapillaryCap.MP4""",
        "recording_type": "camera",
        "ground_truth_lab_notes": """# Change source: UltraSource to ESI source\n\n\n## Aim\nSwitching from the UltraSource to the ESI source.\n\n\n## Materials\n\n### Equipment\n- timsTOF Ultra Mass Spectrometer:\n  - Equipped with an UltraSource ion source\n  - ESI source on the side to attach\n\n\n## Procedure\n*Timing: less than 10 minute*\n\n### Switch TimsControl to 'Standby' mode\n1. Verified that the IonOpticks column from an Evosep System was disconnected.\n2. In timsControl, verified that the software was in standby mode.\n3. Navigated to the 'Source', then proceeded to 'SourceType' and selected 'ESI'. Did not activate the source at this moment.\n\n### Remove UltraSource\n4. Disconnected the power supply of the oven.\n5. Disconnected the filter tubing.\n6. Rotated the top right and bottom left handles of the UltraSource housing by 180 degrees.\n7. Slid off the UltraSource housing from both the source door and the glass capillary.\n8. Set the UltraSource housing on the bench.\n9. Removed the source door by opening it and unhinging it, similar to the motion of a normal door.\n10. Set it aside.\n\n### Mount Apollo ESI source\n11. ❌ **Omitted:** Did not put on gloves.\n12. ❌ **Omitted:** Did not attach the capillary cap securely to the glass capillary.\n13. Attached the spray shield to the desolvation stage housing.\n14. Hinged the ESI source\xa0housing into position as if it were a door and closed it.\n15. Connected the sample inlet of the ESI source and the peak tubing lines, which originated from the syringe, by turning it clockwise.\n16. Connected the nebulizer gas inlet to the N₂ line.\n\n### Prepare the setup by loading the syringe with Tuning Mix\n17. Removed old solvent.\n18. Withdrew new Tuning Mix liquid.\n19. Ensured it was devoid of air bubbles.\n20. Connected the syringe to the peak tubing of the sample line.\n21. Mounted this syringe within the external syringe pump setup. The golden button allowed movement of the syringe holder to arrange the syringe as in Figure 3.\n22. Pressed some solvent out of the syringe to fill the sample line.\n\n### Switch TimsControl to 'Operate' mode\n23. Activated the ESI source within the TimsControl software.\n24. Transitioned the instrument into 'operate' mode by clicking on the on/off symbol.\n25. Began flow at the syringe, ensuring that the following settings were in place in the tab 'Source': Syringe: Hamilton 500 µL; Flow Rate: 3 µL/min. Pressed start. Kept the button next to start/stop pressed until a signal was observed in the 'Chromatogram View' and 'TIMS View' windows, which happened within 30 seconds.\n\n\n\n## Results\n- Signal intensity reached approximately 1.5x10^7\n- Stable signal was observed in timsControl (in the windows for Mobilogram (timsView) and mass spectrum)\n""",
        "error_dict": """[
{"Step": 1, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 2, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 3, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 4, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 5, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 6, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 7, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 8, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 9, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 10, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 11, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 12, "Benchmark": "Error", "Class": "Omitted", "Skill": "ProteomicsKnowledge"},
{"Step": 13, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 14, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 15, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 16, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 17, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 18, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 19, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 20, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 21, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 22, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 23, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 24, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 25, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"}
]""",
        "comments": "Forgot capillary cap",
    },
    {
        "eval_set_name": "DisconnectingColumn_docuCorrect",
        "protocol": '# Disconnecting a IonOpticks column from an Evosep System\n\n## Abstract\nThis protocol describes how to disconnect an IonOpticks column from an Evosep liquid chromatography system when the column is installed in the UltraSource of a timsTOF mass spectrometer.\n\n\n## Materials\n\n### Equipment\n- IonOpticks Column\n- timsTOF Mass Spectrometer:\n  - Equipped with UltraSource ion source\n- Evosep One LC System\n  - with sample line\n- NanoViper Adapter (black)\n- Pliers\n\n\n## Procedure\n*Estimated timing: 1 minute*\n\n### Verify timsTOF is in standby mode:\n1. Check timsControl software status in top left corner.\n2. If in "Operate" mode, click the power symbol to transition to "Standby" (Figure 1, top left).\n\n### Disconnect IonOpticks column and Evosep:\n3. Open the lid of the column oven (column toaster).\n4. Loosen the securing screw at the sample line - column connection (Figure 2F), which has the purpose to establish proper ESI spray grounding with the column oven. Lift it up and move the metal grounding screw away from the column-sample line connection.\n5. Attach the NanoViper adapter to the sample line for easier handling.\n6. Hold the column fitting with pliers for easier handling.\n7. Unscrew the NanoViper connector to detach the sample line from the IonOpticks column by turning it counter clock-wise.\n8. Position the sample line ensuring it does not bend. One method is to place it over the transparent bumper of the Evosep.\n\n\n## Expected Results\n- The timsTOF is in standby mode\n- The column should be completely detached from the Evosep\n\n\n## Figures\n### Figure 1: TimsControl\n\n### Figure 2: IonOpticks column connected with UltraSource of timsTOF and Evosep\n\n## References\n1. Bruker Daltonics. (2023). timsTOF User Manual. [Link to user manual]\n2. Skowronek, P., Wallmann, G., Wahle, M. et al. An accessible workflow for high-sensitivity proteomics using parallel accumulation–serial fragmentation (PASEF). Nat Protoc (2025). https://doi.org/10.1038/s41596-024-01104-w\n',
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/DisconnectingColumn_docuCorrect.MP4""",
        "recording_type": "camera",
        "ground_truth_lab_notes": """# Disconnecting a IonOpticks column from an Evosep System\n\n## Aim\nDisconnecting an IonOpticks column from an Evosep liquid chromatography system while the column is installed in the UltraSource of a timsTOF mass spectrometer.\n\n\n## Materials\n\n### Equipment\n- IonOpticks Column\n- timsTOF Mass Spectrometer:\n  - Equipped with UltraSource ion source\n- Evosep One LC System\n  - with sample line\n- NanoViper Adapter (black)\n- Pliers\n\n\n## Procedure\n*Timing: 1 minute*\n\n### Verified timsTOF was in standby mode:\n1. Checked timsControl software status in top left corner.\n2. Since it was in "Operate" mode, clicked the power symbol to transition to "Standby".\n\n### Disconnected IonOpticks column and Evosep:\n3. Opened the lid of the column oven.\n4. Loosened the securing screw at the sample line - column connection by lifting it up and moved the metal grounding screw away from the column-sample line connection.\n5. Attached the NanoViper adapter to the sample line for easier handling.\n6. Held the column fitting with pliers for easier handling.\n7. Unscrewed the NanoViper connector to detach the sample line from the IonOpticks column by turning it counter clock-wise.\n8. Positioned the sample line without bending it by placing it over the transparent bumper of the Evosep.\n\n\n## Results\n- The timsTOF remained in standby mode\n- The column was completely detached from the Evosep\n""",
        "error_dict": """[
{"Step": 1, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 2, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 3, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 4, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 5, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 6, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 7, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 8, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"}
]""",
        "comments": "",
    },
    {
        "eval_set_name": "DisconnectingColumn_docuWithoutStandby",
        "protocol": '# Disconnecting a IonOpticks column from an Evosep System\n\n## Abstract\nThis protocol describes how to disconnect an IonOpticks column from an Evosep liquid chromatography system when the column is installed in the UltraSource of a timsTOF mass spectrometer.\n\n\n## Materials\n\n### Equipment\n- IonOpticks Column\n- timsTOF Mass Spectrometer:\n  - Equipped with UltraSource ion source\n- Evosep One LC System\n  - with sample line\n- NanoViper Adapter (black)\n- Pliers\n\n\n## Procedure\n*Estimated timing: 1 minute*\n\n### Verify timsTOF is in standby mode:\n1. Check timsControl software status in top left corner.\n2. If in "Operate" mode, click the power symbol to transition to "Standby" (Figure 1, top left).\n\n### Disconnect IonOpticks column and Evosep:\n3. Open the lid of the column oven (column toaster).\n4. Loosen the securing screw at the sample line - column connection (Figure 2F), which has the purpose to establish proper ESI spray grounding with the column oven. Lift it up and move the metal grounding screw away from the column-sample line connection.\n5. Attach the NanoViper adapter to the sample line for easier handling.\n6. Hold the column fitting with pliers for easier handling.\n7. Unscrew the NanoViper connector to detach the sample line from the IonOpticks column by turning it counter clock-wise.\n8. Position the sample line ensuring it does not bend. One method is to place it over the transparent bumper of the Evosep.\n\n\n## Expected Results\n- The timsTOF is in standby mode\n- The column should be completely detached from the Evosep\n\n\n## Figures\n### Figure 1: TimsControl\n\n### Figure 2: IonOpticks column connected with UltraSource of timsTOF and Evosep\n\n## References\n1. Bruker Daltonics. (2023). timsTOF User Manual. [Link to user manual]\n2. Skowronek, P., Wallmann, G., Wahle, M. et al. An accessible workflow for high-sensitivity proteomics using parallel accumulation–serial fragmentation (PASEF). Nat Protoc (2025). https://doi.org/10.1038/s41596-024-01104-w\n',
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/DisconnectingColumn_docuWithoutStandby_1.MP4""",
        "recording_type": "camera",
        "ground_truth_lab_notes": """# Disconnecting a IonOpticks column from an Evosep System\n\n## Aim\nDisconnecting an IonOpticks column from an Evosep liquid chromatography system while the column is installed in the UltraSource of a timsTOF mass spectrometer.\n\n\n## Materials\n\n### Equipment\n- IonOpticks Column\n- timsTOF Mass Spectrometer:\n  - Equipped with UltraSource ion source\n- Evosep One LC System\n  - with sample line\n- NanoViper Adapter (black)\n- Pliers\n\n\n## Procedure\n*Timing: 1 minute*\n\n### Verified timsTOF was in standby mode:\n1. ❌ **Omitted:** Did not check the timsControl software status in top left corner.\n2. ❌ **Omitted:** Skipped to click the power symbol to transition from "Operate" mode to "Standby" mode.\n\n### Disconnected IonOpticks column and Evosep:\n3. Opened the lid of the column oven.\n4. Loosened the securing screw at the sample line - column connection by lifting it up and moved the metal grounding screw away from the column-sample line connection.\n5. Attached the NanoViper adapter to the sample line for easier handling.\n6. Held the column fitting with pliers for easier handling.\n7. Unscrewed the NanoViper connector to detach the sample line from the IonOpticks column by turning it counter clock-wise.\n8. Positioned the sample line without bending it by placing it over the transparent bumper of the Evosep.\n\n\n## Results\n- The timsTOF status, "Operate" or "Standby", is unclear\n- The column was completely detached from the Evosep\n""",
        "error_dict": """[
{"Step": 1, "Benchmark": "Error", "Class": "Omitted", "Skill": "Fast"},
{"Step": 2, "Benchmark": "Error", "Class": "Omitted", "Skill": "ProteomicsKnowledge"},
{"Step": 3, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 4, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 5, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 6, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 7, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 8, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"}
]""",
        "comments": "Did not verify if in standby",
    },
    {
        "eval_set_name": "TimsCalibration_docuCorrect",
        "protocol": "# Calibrate the TIMS device\n\n\n## Abstract\nThis protocol explains how to calibrate a TIMS device in timsControl.\nImportant: We recommend to calibrate the TIMS device each time before a sample queue is started.\n\n\n## Materials\n\n### Software\n| Software | Version | Purpose | Source/Link |\n|----------|---------|---------|------------|\n| timsControl | 6.0 (latest) | Software to control timsTof mass spectrometers | Available on every instrument computer |\n\n\n## Procedure\n*Estimated timing: 3 minutes*\n\nCritical step: If the instrument has been operated with another source than the UltraSource or CaptiveSpray source, it is highly recommended to wait for 3 hours before initiating the TIMS calibration process. This waiting period permits the temperature and pressure to stabilise, thus ensuring a steady TIMS calibration.\n1. Use timsControl to load a performance evaluation method - for instance a dda-PASEF or dia-PASEF method. The method should have an ion mobility range that matches the ion mobility range of the method intended to be use in the study. For proteomics experiments, we typically use an ion mobility range from 0.7 to 1.3 1/K₀, hence the 1/K₀ start and end values can be set to 0.7 and 1.3, respectively (Figure 1: 6, 7). This consistency in ion mobility range allows for the integration of quality control (QC) runs into the sample table as reference points without necessitating recalibration of the TIMS tunnel.\n2. To specifically calibrate a narrow ion mobility range, activate the locked sign at 1/K₀ end (Figure 1: 7).\n3. Adjust the 1/K₀ start from 0.7 to 0.85. This alters the ion mobility range from 0.7-1.3 1/K₀ to 0.85-1.45 1/K₀. The aim here is to shift the ion mobility range without modifying the interval, enabling all three calibrants to be used for linear calibration while maintaining a constant TIMS potential.\n4. Wait until the TIC in Chromatogram View is stable. This can take up to 15 minutes.\n5. Switch the scan mode to 'MS', set MS averaging to 30 and deactivate the polygon heatmap (Figure 1: 4, 9, 14).\n6. In timsControl, navigate to 'calibration', then 'mobility'.\n7. **Not included in video** From reference lists, select the list '[ESI] Tuning Mix ES-TOF (ESI)' that contains the calibrant masses 622, 922, 1221.\n8. Specify the linear mode and 5% as detection range and ±0.1 Da as width (Figure 2).\n9. Proceed by selecting 'calibrate' (Figure 2).\n10. To verify that the calibrants have been picked correctly at the center, click on them in the reference list.\n11. If they are not picked in the center, make adjustments by clicking on the peak in the TIMS view window.\n12. If the score is at 100%, press accept.\n13. Select \"Method\" > \"Load Recent\", select the same method, and then click \"Discard changes\" in the pop-up window.\n14. Set MS averaging to 1.\n\n\n## Expected Results\n- The score in the tab \"Calibration\" should be at 100%\n\n\n## Figures\n### Figure 1: TimsControl settings\n\n### Figure 2: Ion mobility calibration\n\n## Tables\nTable 1: Gas Flow Parameters for UltraSource\n\n| Instrument Type | Calibrant 922 Voltage [V] |\n|----------------|---------------------------|\n| timsTOF Pro, SCP | 160 |\n| timsTOF HT, Ultra | 200 |\n\n## References\n1. Skowronek, P., Wallmann, G., Wahle, M. et al. An accessible workflow for high-sensitivity proteomics using parallel accumulation–serial fragmentation (PASEF). Nat Protoc (2025). https://doi.org/10.1038/s41596-024-01104-w\n2. TODO: Refer to user manual of timsTof\n3. TODO: Maybe upload video which shows calibration in action\n",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/TimsCalibration_docuCorrect.mov""",
        "recording_type": "screen_recording",
        "ground_truth_lab_notes": """# Calibrate the TIMS device\n\n\n## Aim\nCalibrating a TIMS device in timsControl.\n\n\n## Materials\n\n### Software\ntimsControl 6.0\n\n\n## Procedure\n*Timing: 1.5 minutes*\n\n1. Verified that dia-PASEF method had an ion mobility range that matched the ion mobility range of the method intended to be used in the proteomics study: ion mobility range from 0.7 to 1.3 1/K₀. Hence the 1/K₀ start and end values were set to 0.7 and 1.3, respectively.\n2. Activated the locked sign at 1/K₀ end.\n3. Adjusted the 1/K₀ start from 0.7 to 0.85. This altered the ion mobility range from 0.7-1.3 1/K₀ to 0.85-1.45 1/K₀.\n4. Waited until the TIC in Chromatogram View was stable. This took approximately 1 minute.\n5. Switched the scan mode to \'MS\', set MS averaging to 30 and kept the polygon heatmap deactivated.\n6. In timsControl, navigated to \'calibration\', then \'mobility\'.\n7. Verified that the reference list \'[ESI] Tuning Mix ES-TOF (ESI)\' is selected that contains the calibrant masses 622, 922, 1221.\n8. Verified the linear mode and 5% as detection range and ±0.1 Da as width.\n9. Proceeded by selecting \'calibrate\'.\n10. To verify that the calibrants had been picked correctly at the center, clicked on them in the reference list.\n11. Light blue peak (corresponding to a m/z at 1222) was not picked in the center, made adjustments by clicking on the peak in the TIMS view window.\n12. When the score reached 100%, pressed accept.\n13. Selected "Method" > "Load Recent", selected the same method, and then clicked "Discard changes" in the pop-up window.\n14. Set MS averaging back to 1.\n\n\n## Results\n- The score in the tab "Calibration" was at 100%\n""",
        "error_dict": """[
  {"Step": 1, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 2, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 3, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 4, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 5, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 6, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 7, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 8, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 9, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 10, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 11, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 12, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 13, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 14, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"}
]""",
        "comments": "",
    },
    {
        "eval_set_name": "TimsCalibration_docuCorrect_camera",
        "protocol": "# Calibrate the TIMS device\n\n\n## Abstract\nThis protocol explains how to calibrate a TIMS device in timsControl.\nImportant: We recommend to calibrate the TIMS device each time before a sample queue is started.\n\n\n## Materials\n\n### Software\n| Software | Version | Purpose | Source/Link |\n|----------|---------|---------|------------|\n| timsControl | 6.0 (latest) | Software to control timsTof mass spectrometers | Available on every instrument computer |\n\n\n## Procedure\n*Estimated timing: 3 minutes*\n\nCritical step: If the instrument has been operated with another source than the UltraSource or CaptiveSpray source, it is highly recommended to wait for 3 hours before initiating the TIMS calibration process. This waiting period permits the temperature and pressure to stabilise, thus ensuring a steady TIMS calibration.\n1. Use timsControl to load a performance evaluation method - for instance a dda-PASEF or dia-PASEF method. The method should have an ion mobility range that matches the ion mobility range of the method intended to be use in the study. For proteomics experiments, we typically use an ion mobility range from 0.7 to 1.3 1/K₀, hence the 1/K₀ start and end values can be set to 0.7 and 1.3, respectively (Figure 1: 6, 7). This consistency in ion mobility range allows for the integration of quality control (QC) runs into the sample table as reference points without necessitating recalibration of the TIMS tunnel.\n2. To specifically calibrate a narrow ion mobility range, activate the locked sign at 1/K₀ end (Figure 1: 7).\n3. Adjust the 1/K₀ start from 0.7 to 0.85. This alters the ion mobility range from 0.7-1.3 1/K₀ to 0.85-1.45 1/K₀. The aim here is to shift the ion mobility range without modifying the interval, enabling all three calibrants to be used for linear calibration while maintaining a constant TIMS potential.\n4. Wait until the TIC in Chromatogram View is stable. This can take up to 15 minutes.\n5. Switch the scan mode to 'MS', set MS averaging to 30 and deactivate the polygon heatmap (Figure 1: 4, 9, 14).\n6. In timsControl, navigate to 'calibration', then 'mobility'.\n7. **Not included in video** From reference lists, select the list '[ESI] Tuning Mix ES-TOF (ESI)' that contains the calibrant masses 622, 922, 1221.\n8. Specify the linear mode and 5% as detection range and ±0.1 Da as width (Figure 2).\n9. Proceed by selecting 'calibrate' (Figure 2).\n10. To verify that the calibrants have been picked correctly at the center, click on them in the reference list.\n11. If they are not picked in the center, make adjustments by clicking on the peak in the TIMS view window.\n12. If the score is at 100%, press accept.\n13. Select \"Method\" > \"Load Recent\", select the same method, and then click \"Discard changes\" in the pop-up window.\n14. Set MS averaging to 1.\n\n\n## Expected Results\n- The score in the tab \"Calibration\" should be at 100%\n\n\n## Figures\n### Figure 1: TimsControl settings\n\n### Figure 2: Ion mobility calibration\n\n## Tables\nTable 1: Gas Flow Parameters for UltraSource\n\n| Instrument Type | Calibrant 922 Voltage [V] |\n|----------------|---------------------------|\n| timsTOF Pro, SCP | 160 |\n| timsTOF HT, Ultra | 200 |\n\n## References\n1. Skowronek, P., Wallmann, G., Wahle, M. et al. An accessible workflow for high-sensitivity proteomics using parallel accumulation–serial fragmentation (PASEF). Nat Protoc (2025). https://doi.org/10.1038/s41596-024-01104-w\n2. TODO: Refer to user manual of timsTof\n3. TODO: Maybe upload video which shows calibration in action\n",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/TimsCalibration_docuCorrect_camera.MP4""",
        "recording_type": "camera",
        "ground_truth_lab_notes": """# Calibrate the TIMS device\n\n\n## Aim\nCalibrating a TIMS device in timsControl.\n\n\n## Materials\n\n### Software\ntimsControl 6.0\n\n\n## Procedure\n*Timing: 1.5 minutes*\n\n1. Verified that dia-PASEF method had an ion mobility range that matched the ion mobility range of the method intended to be used in the proteomics study: ion mobility range from 0.7 to 1.3 1/K₀. Hence the 1/K₀ start and end values were set to 0.7 and 1.3, respectively.\n2. Activated the locked sign at 1/K₀ end.\n3. Adjusted the 1/K₀ start from 0.7 to 0.85. This altered the ion mobility range from 0.7-1.3 1/K₀ to 0.85-1.45 1/K₀.\n4. Waited until the TIC in Chromatogram View was stable. This took approximately 1 minute.\n5. Switched the scan mode to \'MS\', set MS averaging to 30 and kept the polygon heatmap deactivated.\n6. In timsControl, navigated to \'calibration\', then \'mobility\'.\n7. Verified that the reference list \'[ESI] Tuning Mix ES-TOF (ESI)\' is selected that contains the calibrant masses 622, 922, 1221.\n8. Verified the linear mode and 5% as detection range and ±0.1 Da as width.\n9. Proceeded by selecting \'calibrate\'.\n10. To verify that the calibrants had been picked correctly at the center, clicked on them in the reference list.\n11. Light blue peak (corresponding to a m/z at 1222) was not picked in the center, made adjustments by clicking on the peak in the TIMS view window.\n12. When the score reached 100%, pressed accept.\n13. Selected "Method" > "Load Recent", selected the same method, and then clicked "Discard changes" in the pop-up window.\n14. Set MS averaging back to 1.\n\n\n## Results\n- The score in the tab "Calibration" was at 100%\n""",
        "error_dict": """[
  {"Step": 1, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 2, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 3, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 4, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 5, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 6, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 7, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 8, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 9, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 10, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 11, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 12, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 13, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 14, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"}
]""",
        "comments": "",
    },
    {
        "eval_set_name": "TimsCalibration_docuNotAllClicksVisibleOnVideo",
        "protocol": "# Calibrate the TIMS device\n\n\n## Abstract\nThis protocol explains how to calibrate a TIMS device in timsControl.\nImportant: We recommend to calibrate the TIMS device each time before a sample queue is started.\n\n\n## Materials\n\n### Software\n| Software | Version | Purpose | Source/Link |\n|----------|---------|---------|------------|\n| timsControl | 6.0 (latest) | Software to control timsTof mass spectrometers | Available on every instrument computer |\n\n\n## Procedure\n*Estimated timing: 3 minutes*\n\nCritical step: If the instrument has been operated with another source than the UltraSource or CaptiveSpray source, it is highly recommended to wait for 3 hours before initiating the TIMS calibration process. This waiting period permits the temperature and pressure to stabilise, thus ensuring a steady TIMS calibration.\n1. Use timsControl to load a performance evaluation method - for instance a dda-PASEF or dia-PASEF method. The method should have an ion mobility range that matches the ion mobility range of the method intended to be use in the study. For proteomics experiments, we typically use an ion mobility range from 0.7 to 1.3 1/K₀, hence the 1/K₀ start and end values can be set to 0.7 and 1.3, respectively (Figure 1: 6, 7). This consistency in ion mobility range allows for the integration of quality control (QC) runs into the sample table as reference points without necessitating recalibration of the TIMS tunnel.\n2. To specifically calibrate a narrow ion mobility range, activate the locked sign at 1/K₀ end (Figure 1: 7).\n3. Adjust the 1/K₀ start from 0.7 to 0.85. This alters the ion mobility range from 0.7-1.3 1/K₀ to 0.85-1.45 1/K₀. The aim here is to shift the ion mobility range without modifying the interval, enabling all three calibrants to be used for linear calibration while maintaining a constant TIMS potential.\n4. Wait until the TIC in Chromatogram View is stable. This can take up to 15 minutes.\n5. Switch the scan mode to 'MS', set MS averaging to 30 and deactivate the polygon heatmap (Figure 1: 4, 9, 14).\n6. In timsControl, navigate to 'calibration', then 'mobility'.\n7. **Not included in video** From reference lists, select the list '[ESI] Tuning Mix ES-TOF (ESI)' that contains the calibrant masses 622, 922, 1221.\n8. Specify the linear mode and 5% as detection range and ±0.1 Da as width (Figure 2).\n9. Proceed by selecting 'calibrate' (Figure 2).\n10. To verify that the calibrants have been picked correctly at the center, click on them in the reference list.\n11. If they are not picked in the center, make adjustments by clicking on the peak in the TIMS view window.\n12. If the score is at 100%, press accept.\n13. Select \"Method\" > \"Load Recent\", select the same method, and then click \"Discard changes\" in the pop-up window.\n14. Set MS averaging to 1.\n\n\n## Expected Results\n- The score in the tab \"Calibration\" should be at 100%\n\n\n## Figures\n### Figure 1: TimsControl settings\n\n### Figure 2: Ion mobility calibration\n\n## Tables\nTable 1: Gas Flow Parameters for UltraSource\n\n| Instrument Type | Calibrant 922 Voltage [V] |\n|----------------|---------------------------|\n| timsTOF Pro, SCP | 160 |\n| timsTOF HT, Ultra | 200 |\n\n## References\n1. Skowronek, P., Wallmann, G., Wahle, M. et al. An accessible workflow for high-sensitivity proteomics using parallel accumulation–serial fragmentation (PASEF). Nat Protoc (2025). https://doi.org/10.1038/s41596-024-01104-w\n2. TODO: Refer to user manual of timsTof\n3. TODO: Maybe upload video which shows calibration in action\n",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/TimsCalibration_docuNotAllClicksVisibleOnVideo.MP4""",
        "recording_type": "camera",
        "ground_truth_lab_notes": """# Calibrate the TIMS device\n\n\n## Aim\nCalibrating a TIMS device in timsControl.\n\n\n## Materials\n\n### Software\ntimsControl 6.0\n\n\n## Procedure\n*Timing: 1.5 minutes*\n\n1. Verified that dia-PASEF method had an ion mobility range that matched the ion mobility range of the method intended to be used in the proteomics study: ion mobility range from 0.7 to 1.3 1/K₀. Hence the 1/K₀ start and end values were set to 0.7 and 1.3, respectively.\n2. Activated the locked sign at 1/K₀ end.\n3. Adjusted the 1/K₀ start from 0.7 to 0.85. This altered the ion mobility range from 0.7-1.3 1/K₀ to 0.85-1.45 1/K₀.\n4. Waited until the TIC in Chromatogram View was stable. This took approximately 1 minute.\n5. Switched the scan mode to \'MS\', set MS averaging to 30 and kept the polygon heatmap deactivated.\n6. In timsControl, navigated to \'calibration\', then \'mobility\'.\n7. Verified that the reference list \'[ESI] Tuning Mix ES-TOF (ESI)\' is selected that contains the calibrant masses 622, 922, 1221.\n8. Verified the linear mode and 5% as detection range and ±0.1 Da as width.\n9. Proceeded by selecting \'calibrate\'.\n10. To verify that the calibrants had been picked correctly at the center, clicked on them in the reference list.\n11. No adjustments by clicking on the peak in the TIMS view window were necessary.\n12. When the score reached 100%, pressed accept.\n13. **Cursor clicks not on video: Selected "Method" > "Load Recent", selected the same method** and then clicked "Discard changes" in the pop-up window.\n14. Set MS averaging back to 1.\n\n\n## Results\n- The score in the tab "Calibration" was at 100%\n""",
        "error_dict": """[
  {"Step": 1, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 2, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 3, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 4, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 5, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 6, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 7, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 8, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 9, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 10, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 11, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 12, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 13, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 14, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"}
]""",
        "comments": "Procedure is correct but clicks regarding step 12 “Tried to save the method” are visible on video.",
    },
    {
        "eval_set_name": "TimsCalibration_docuSavedMethod",
        "protocol": "# Calibrate the TIMS device\n\n\n## Abstract\nThis protocol explains how to calibrate a TIMS device in timsControl.\nImportant: We recommend to calibrate the TIMS device each time before a sample queue is started.\n\n\n## Materials\n\n### Software\n| Software | Version | Purpose | Source/Link |\n|----------|---------|---------|------------|\n| timsControl | 6.0 (latest) | Software to control timsTof mass spectrometers | Available on every instrument computer |\n\n\n## Procedure\n*Estimated timing: 3 minutes*\n\nCritical step: If the instrument has been operated with another source than the UltraSource or CaptiveSpray source, it is highly recommended to wait for 3 hours before initiating the TIMS calibration process. This waiting period permits the temperature and pressure to stabilise, thus ensuring a steady TIMS calibration.\n1. Use timsControl to load a performance evaluation method - for instance a dda-PASEF or dia-PASEF method. The method should have an ion mobility range that matches the ion mobility range of the method intended to be use in the study. For proteomics experiments, we typically use an ion mobility range from 0.7 to 1.3 1/K₀, hence the 1/K₀ start and end values can be set to 0.7 and 1.3, respectively (Figure 1: 6, 7). This consistency in ion mobility range allows for the integration of quality control (QC) runs into the sample table as reference points without necessitating recalibration of the TIMS tunnel.\n2. To specifically calibrate a narrow ion mobility range, activate the locked sign at 1/K₀ end (Figure 1: 7).\n3. Adjust the 1/K₀ start from 0.7 to 0.85. This alters the ion mobility range from 0.7-1.3 1/K₀ to 0.85-1.45 1/K₀. The aim here is to shift the ion mobility range without modifying the interval, enabling all three calibrants to be used for linear calibration while maintaining a constant TIMS potential.\n4. Wait until the TIC in Chromatogram View is stable. This can take up to 15 minutes.\n5. Switch the scan mode to 'MS', set MS averaging to 30 and deactivate the polygon heatmap (Figure 1: 4, 9, 14).\n6. In timsControl, navigate to 'calibration', then 'mobility'.\n7. **Not included in video** From reference lists, select the list '[ESI] Tuning Mix ES-TOF (ESI)' that contains the calibrant masses 622, 922, 1221.\n8. Specify the linear mode and 5% as detection range and ±0.1 Da as width (Figure 2).\n9. Proceed by selecting 'calibrate' (Figure 2).\n10. To verify that the calibrants have been picked correctly at the center, click on them in the reference list.\n11. If they are not picked in the center, make adjustments by clicking on the peak in the TIMS view window.\n12. If the score is at 100%, press accept.\n13. Select \"Method\" > \"Load Recent\", select the same method, and then click \"Discard changes\" in the pop-up window.\n14. Set MS averaging to 1.\n\n\n## Expected Results\n- The score in the tab \"Calibration\" should be at 100%\n\n\n## Figures\n### Figure 1: TimsControl settings\n\n### Figure 2: Ion mobility calibration\n\n## Tables\nTable 1: Gas Flow Parameters for UltraSource\n\n| Instrument Type | Calibrant 922 Voltage [V] |\n|----------------|---------------------------|\n| timsTOF Pro, SCP | 160 |\n| timsTOF HT, Ultra | 200 |\n\n## References\n1. Skowronek, P., Wallmann, G., Wahle, M. et al. An accessible workflow for high-sensitivity proteomics using parallel accumulation–serial fragmentation (PASEF). Nat Protoc (2025). https://doi.org/10.1038/s41596-024-01104-w\n2. TODO: Refer to user manual of timsTof\n3. TODO: Maybe upload video which shows calibration in action\n",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/TimsCalibration_docuSavedMethod.mov""",
        "recording_type": "screen_recording",
        "ground_truth_lab_notes": """# Calibrate the TIMS device\n\n\n## Aim\nCalibrating a TIMS device in timsControl.\n\n\n## Materials\n\n### Software\ntimsControl 6.0\n\n\n## Procedure\n*Timing: 1.5 minutes*\n\n1. Verified that dia-PASEF method had an ion mobility range that matched the ion mobility range of the method intended to be used in the proteomics study: ion mobility range from 0.7 to 1.3 1/K₀. Hence the 1/K₀ start and end values were set to 0.7 and 1.3, respectively.\n2. Activated the locked sign at 1/K₀ end.\n3. Adjusted the 1/K₀ start from 0.7 to 0.85. This altered the ion mobility range from 0.7-1.3 1/K₀ to 0.85-1.45 1/K₀.\n4. Waited until the TIC in Chromatogram View was stable. This took approximately 1 minute.\n5. Switched the scan mode to \'MS\', set MS averaging to 30 and kept the polygon heatmap deactivated .\n6. In timsControl, navigated to \'calibration\', then \'mobility\'.\n7. Verified that the reference list \'[ESI] Tuning Mix ES-TOF (ESI)\' is selected that contains the calibrant masses 622, 922, 1221.\n8. Verified the linear mode and 5% as detection range and ±0.1 Da as width.\n9. Proceeded by selecting \'calibrate\'.\n10. To verify that the calibrants had been picked correctly at the center, clicked on them in the reference list.\n12. ⚠️ **Deviation: Altered step order** When the score reached 100%, pressed accept.\n12.1 ➕ **Added:** Repeated step 8 and 9.\n11. Turquiouse peak (corresponding to a m/z at 922) was not picked in the center, made adjustments by clicking on the peak in the TIMS view window.\n12.2 When the score reached 100%, pressed accept.\n13. ❌ **Error:** Saved the method instead of "Discard all changes".\n14. ❌ **Omitted:** Did not set MS averaging back to 1.\n\n\n## Results\n- ✓ The score in the tab "Calibration" was at 100%\n""",
        "error_dict": """[
  {"Step": 1, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 2, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 3, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 4, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 5, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 6, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 7, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 8, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 9, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 10, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 11, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 12, "Benchmark": "Error", "Class": "Deviation", "Skill": "ProteomicsKnowledge"},
  {"Step": 12.1, "Benchmark": "Error", "Class": "Addition", "Skill": "ProteomicsKnowledge"},
  {"Step": 12.2, "Benchmark": "Error", "Class": "Addition", "Skill": "ProteomicsKnowledge"},
  {"Step": 13, "Benchmark": "Error", "Class": "Error", "Skill": "ProteomicsKnowledge"},
  {"Step": 14, "Benchmark": "Error", "Class": "Omitted", "Skill": "ProteomicsKnowledge"}
]""",
        "comments": "method saved instead of discarded, did not set back to MS averaging 1",
    },
    {
        "eval_set_name": "TimsCalibration_docuWrongOrderSteps",
        "protocol": "# Calibrate the TIMS device\n\n\n## Abstract\nThis protocol explains how to calibrate a TIMS device in timsControl.\nImportant: We recommend to calibrate the TIMS device each time before a sample queue is started.\n\n\n## Materials\n\n### Software\n| Software | Version | Purpose | Source/Link |\n|----------|---------|---------|------------|\n| timsControl | 6.0 (latest) | Software to control timsTof mass spectrometers | Available on every instrument computer |\n\n\n## Procedure\n*Estimated timing: 3 minutes*\n\nCritical step: If the instrument has been operated with another source than the UltraSource or CaptiveSpray source, it is highly recommended to wait for 3 hours before initiating the TIMS calibration process. This waiting period permits the temperature and pressure to stabilise, thus ensuring a steady TIMS calibration.\n1. Use timsControl to load a performance evaluation method - for instance a dda-PASEF or dia-PASEF method. The method should have an ion mobility range that matches the ion mobility range of the method intended to be use in the study. For proteomics experiments, we typically use an ion mobility range from 0.7 to 1.3 1/K₀, hence the 1/K₀ start and end values can be set to 0.7 and 1.3, respectively (Figure 1: 6, 7). This consistency in ion mobility range allows for the integration of quality control (QC) runs into the sample table as reference points without necessitating recalibration of the TIMS tunnel.\n2. To specifically calibrate a narrow ion mobility range, activate the locked sign at 1/K₀ end (Figure 1: 7).\n3. Adjust the 1/K₀ start from 0.7 to 0.85. This alters the ion mobility range from 0.7-1.3 1/K₀ to 0.85-1.45 1/K₀. The aim here is to shift the ion mobility range without modifying the interval, enabling all three calibrants to be used for linear calibration while maintaining a constant TIMS potential.\n4. Wait until the TIC in Chromatogram View is stable. This can take up to 15 minutes.\n5. Switch the scan mode to 'MS', set MS averaging to 30 and deactivate the polygon heatmap (Figure 1: 4, 9, 14).\n6. In timsControl, navigate to 'calibration', then 'mobility'.\n7. **Not included in video** From reference lists, select the list '[ESI] Tuning Mix ES-TOF (ESI)' that contains the calibrant masses 622, 922, 1221.\n8. Specify the linear mode and 5% as detection range and ±0.1 Da as width (Figure 2).\n9. Proceed by selecting 'calibrate' (Figure 2).\n10. To verify that the calibrants have been picked correctly at the center, click on them in the reference list.\n11. If they are not picked in the center, make adjustments by clicking on the peak in the TIMS view window.\n12. If the score is at 100%, press accept.\n13. Select \"Method\" > \"Load Recent\", select the same method, and then click \"Discard changes\" in the pop-up window.\n14. Set MS averaging to 1.\n\n\n## Expected Results\n- The score in the tab \"Calibration\" should be at 100%\n\n\n## Figures\n### Figure 1: TimsControl settings\n\n### Figure 2: Ion mobility calibration\n\n## Tables\nTable 1: Gas Flow Parameters for UltraSource\n\n| Instrument Type | Calibrant 922 Voltage [V] |\n|----------------|---------------------------|\n| timsTOF Pro, SCP | 160 |\n| timsTOF HT, Ultra | 200 |\n\n## References\n1. Skowronek, P., Wallmann, G., Wahle, M. et al. An accessible workflow for high-sensitivity proteomics using parallel accumulation–serial fragmentation (PASEF). Nat Protoc (2025). https://doi.org/10.1038/s41596-024-01104-w\n2. TODO: Refer to user manual of timsTof\n3. TODO: Maybe upload video which shows calibration in action\n",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/TimsCalibration_docuWrongOrderSteps.mov""",
        "recording_type": "screen_recording",
        "ground_truth_lab_notes": """# Calibrate the TIMS device\n\n\n## Aim\nCalibrating a TIMS device in timsControl.\n\n\n## Materials\n\n### Software\ntimsControl 6.0\n\n\n## Procedure\n*Timing: 1.5 minutes*\n\n1. Verified that dia-PASEF method had an ion mobility range that matched the ion mobility range of the method intended to be used in the proteomics study: ion mobility range from 0.7 to 1.3 1/K₀. Hence the 1/K₀ start and end values were set to 0.7 and 1.3, respectively.\n2. Activated the locked sign at 1/K₀ end.\n3. Adjusted the 1/K₀ start from 0.7 to 0.85. This altered the ion mobility range from 0.7-1.3 1/K₀ to 0.85-1.45 1/K₀.\n6. In timsControl, navigated to \'calibration\', then \'mobility\'.\n7. Verified that the reference list \'[ESI] Tuning Mix ES-TOF (ESI)\' is selected that contains the calibrant masses 622, 922, 1221.\n8. Verified the linear mode and 5% as detection range and ±0.1 Da as width.\n4. ⚠️ **Deviation: Altered step order** Waited until the TIC in Chromatogram View was stable. This took approximately 1 minute.\n9. ❌ **Error:** Proceeded by selecting \'calibrate\' and canceled \'calibrate\'.\n5. ⚠️ **Deviation: Altered step order** Switched the scan mode to \'MS\', set MS averaging to 30 and kept the polygon heatmap deactivated .\n9.2 Proceeded by selecting \'calibrate\'.\n10. To verify that the calibrants had been picked correctly at the center, clicked on them in the reference list.\n11. Turquoise peak (corresponding to a m/z at 922) was not picked in the center, made adjustments by clicking on the peak in the TIMS view window.\n12. When the score reached 100%, pressed accept.\n13. Selected "Method" > "Load Recent", selected the same method, and then clicked "Discard changes" in the pop-up window.\n14. Set MS averaging back to 1.\n\n\n## Results\n- The score in the tab "Calibration" was at 100%\n""",
        "error_dict": """[
{"Step": 1, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 2, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 3, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 4, "Benchmark": "Error", "Class": "Deviation", "Skill": "Fast"},
{"Step": 5, "Benchmark": "Error", "Class": "Deviation", "Skill": "ProteomicsKnowledge"},
{"Step": 6, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 7, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 8, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 9, "Benchmark": "Error", "Class": "Error", "Skill": "ProteomicsKnowledge"},
{"Step": 10, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 11, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 12, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 13, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 14, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"}
]""",
        "comments": "Wrong order of steps → led to adding a step and repeating a step",
    },
    {
        "eval_set_name": "QueueSamples_docuCorrect_1",
        "protocol": """# Queue and measure samples in HyStar\n\n\n## Abstract\nThis protocol explains how to queue sample in HyStar for LC-MS measurement.\n\n\n## Materials\n\n### Software\n| Software | Version | Purpose | Source/Link |\n|----------|---------|---------|------------|\n| HyStar | 6.0 | Controls LC and MS | On every instrument computer |\n\n\n## Procedure\n*Estimated timing: 6 minutes*\n\n1. Verify the Evotips were placed on top of the Evosep LC system. In this example, 5 ng HeLa Evotips were placed at S1 from A1 to A6 and blanks at S3 from A1 to A6.\n    Note: Blank Evotis can be unused and dry Evotips.\n2. Verify that the TIMS device in TimsControl is calibrated. The TIMS device should be calibrated each time before you start a sample queue.\n3. In Hystar, navigate to the \'Acquisition\' tab.\n4. Either select \'New\', and subsequently choose \'LC-MS sample table\' (Figure 1) to generate a new sample table or choose an already existing sample table by pressing the arrow down button when hovering over the sample table name in the left sample table column.\n5. In both cases, copy already existing sample table entries to modify them.\n6. Adjust the sample ID so that it follows this pattern: currentDate_massSpec_user_sampleType_projectID_ sampleName. Typical examples for sampleType: "SA_blank", "MA_HeLa", "DIAMA_HeLa".\n7. For performance evaluation of the LC-MS system, we recommend to queue one blank, three dda-PASEF runs, three dia-PASEF runs and ending with another blank.\n    Critical step: Always start the queue with a sacrificial Evotip such as a blank tip as the first run might have altered chromatographic conditions that could introduce technical errors to your measurements.\n8. Verify the column autocompletion settings with right-click on a field in the column \'vial\' such as S1-A1 in Figure 1 and select \'Configure\'. The arrows allow one to define the direction in which the vial positions on the 96-well are autocompleted when dragging values similar to Excel in the sample table. Decide whether the values should increase from A1-A12 indicated by arrows pointing to right. Ensure that the tray type is set to \'Evosep\' and slots 1-6 are designated as \'96Evotip\'. Press \'OK\'.\n9. Match the Evotip position with the sample\'s location in the Evotip box: From S1 from A1 to A6 and blanks at S3 from A1 to A6. To do this, press the arrow next to the value in the \'vial\' column (Figure 1). Select the position where the first Evotip is placed, for instance S1 A1. You can then either specify all remaining positions automatically by dragging the values (similar to Excel\'s auto-fill function) or specify each position individually.\n10. Specify a \'path\' folder for storing the raw files.\n11. Choose an existing separation method or create a new one. To create a new separation method, right-click on the separation method field, select \'new method\' followed by \'edit method\'. Choose the method type listed under \'name\' and then press \'OK\'. Specify its name and save it.\n12. Set the injection method to \'standard\'.\n13. At \'MS method\', load either dda-PASEF and dia-PASEF maintenance methods to check the LC-MS performance or the method intended to be used for measuring the study.\n14. Stop the idle flow on the Evosep by right-clicking on the Evosep logo and selecting \'Cancel maintenance procedure\'.\n15. Save the sample table.\n16. Right-click somewhere on the top row of the freshly defined sample table entries and select \'upload sample conditions\' to pre-check if all LC and MS method can be loaded correctly. The status should be change to loaded.\n17. Press \'start\' and \'start sequence\'.\n\n\n## Expected Results\nTypical chromatogram shape with MS TIC (green line) increasing drastically around 4 minutes and another increase (wash-out) at 15 minutes. Higher MS2 intensities (red line) for dia-PASEF.\n\n\n## Figures\n### Figure 1: Hystar\n[Placeholder for Screenshot of reaauired Hystar settings]\n\n\n## References\n1. Skowronek, P., Wallmann, G., Wahle, M. et al. An accessible workflow for high-sensitivity proteomics using parallel accumulation–serial fragmentation (PASEF). Nat Protoc (2025). https://doi.org/10.1038/s41596-024-01104-w\n2. TODO: Refer to Hystar manual\n3. TODO: Maybe upload video which shows protocol in action\n""",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/QueueSamples_docuCorrect_1.mov""",
        "recording_type": "screen_recording",
        "ground_truth_lab_notes": """# Queue and measure samples in HyStar\n\n\n## Aim\nQueuing samples in HyStar for LC-MS measurement.\n\n\n## Materials\n\n### Software\nHyStar 6.0\n\n\n## Procedure\n*Timing: 2 minutes*\n\n1. Verified that Evotips are placed on top of Evosep. 5 ng HeLa Evotips at S1 from A1 to A6 and blanks at S3 from A1 to A6.\n2. Verified that the calibration of the TIMS device in TimsControl is valid.\n3. In Hystar, navigated to the 'Acquisition' tab.\n4. Chose an already existing sample table by pressing the arrow down button when hovering over the sample table name in the left sample table column.\n5. Copied already existing sample table entries to modify them.\n6. Adjusted the sample ID so that it followed this pattern: currentDate_massSpec_user_sampleType_projectID_sampleName.\n7. For performance evaluation of the LC-MS system, queued one blank, three dda-PASEF runs, three dia-PASEF runs and ended with another blank.\n8. Verified the column autocompletion settings by right-clicking on a field in the column 'vial' and selected 'Configure'. Values were set to autocomplete from A1-A12 indicated by arrows pointing to right. Ensured that the tray type was set to 'Evosep' and slots 1-6 were designated as '96Evotip'. Pressed 'OK'.\n9. Matched the Evotip position with the sample's location in the Evotip box. To do this, pressed the arrow next to the value in the 'vial' column. Specified blank Evotip position individually. Selected the position where the first sample Evotip was placed (S1 A1). Then automatically specified all remaining positions by dragging the values (similar to Excel's auto-fill function).\n10. Verified 'path' folder for storing the raw files.\n11. Verified separation method.\n12. Verified that injection method is set to 'standard'.\n13. At 'MS method', adjusted dda-PASEF and dia-PASEF maintenance methods according to sample ID.\n14. Stopped the idle flow on the Evosep by right-clicking on the Evosep logo and selecting 'Cancel maintenance procedure'.\n15. Saved the sample table.\n16. Right-clicked on the top row of the freshly defined sample table entries and selected 'upload sample conditions' to pre-check if all LC and MS methods could be loaded correctly. The status changed to loaded.\n17. Pressed 'start' and 'start sequence'.\n\n\n## Results\nChromatogram is not shown in video.\n""",
        "error_dict": """[
{"Step": 1, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 2, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 3, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 4, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 5, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 6, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 7, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 8, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 9, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 10, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 11, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 12, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 13, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 14, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 15, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 16, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 17, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"}
]""",
        "comments": "",
    },
    {
        "eval_set_name": "QueueSamples_docuWrongRow_S3A1Twice",
        "protocol": """# Queue and measure samples in HyStar\n\n\n## Abstract\nThis protocol explains how to queue sample in HyStar for LC-MS measurement.\n\n\n## Materials\n\n### Software\n| Software | Version | Purpose | Source/Link |\n|----------|---------|---------|------------|\n| HyStar | 6.0 | Controls LC and MS | On every instrument computer |\n\n\n## Procedure\n*Estimated timing: 6 minutes*\n\n1. Verify the Evotips were placed on top of the Evosep LC system. In this example, 5 ng HeLa Evotips were placed at S1 from A1 to A6 and blanks at S3 from A1 to A6.\n    Note: Blank Evotis can be unused and dry Evotips.\n2. Verify that the TIMS device in TimsControl is calibrated. The TIMS device should be calibrated each time before you start a sample queue.\n3. In Hystar, navigate to the \'Acquisition\' tab.\n4. Either select \'New\', and subsequently choose \'LC-MS sample table\' (Figure 1) to generate a new sample table or choose an already existing sample table by pressing the arrow down button when hovering over the sample table name in the left sample table column.\n5. In both cases, copy already existing sample table entries to modify them.\n6. Adjust the sample ID so that it follows this pattern: currentDate_massSpec_user_sampleType_projectID_ sampleName. Typical examples for sampleType: "SA_blank", "MA_HeLa", "DIAMA_HeLa".\n7. For performance evaluation of the LC-MS system, we recommend to queue one blank, three dda-PASEF runs, three dia-PASEF runs and ending with another blank.\n    Critical step: Always start the queue with a sacrificial Evotip such as a blank tip as the first run might have altered chromatographic conditions that could introduce technical errors to your measurements.\n8. Verify the column autocompletion settings with right-click on a field in the column \'vial\' such as S1-A1 in Figure 1 and select \'Configure\'. The arrows allow one to define the direction in which the vial positions on the 96-well are autocompleted when dragging values similar to Excel in the sample table. Decide whether the values should increase from A1-A12 indicated by arrows pointing to right. Ensure that the tray type is set to \'Evosep\' and slots 1-6 are designated as \'96Evotip\'. Press \'OK\'.\n9. Match the Evotip position with the sample\'s location in the Evotip box: From S1 from A1 to A6 and blanks at S3 from A1 to A6. To do this, press the arrow next to the value in the \'vial\' column (Figure 1). Select the position where the first Evotip is placed, for instance S1 A1. You can then either specify all remaining positions automatically by dragging the values (similar to Excel\'s auto-fill function) or specify each position individually.\n10. Specify a \'path\' folder for storing the raw files.\n11. Choose an existing separation method or create a new one. To create a new separation method, right-click on the separation method field, select \'new method\' followed by \'edit method\'. Choose the method type listed under \'name\' and then press \'OK\'. Specify its name and save it.\n12. Set the injection method to \'standard\'.\n13. At \'MS method\', load either dda-PASEF and dia-PASEF maintenance methods to check the LC-MS performance or the method intended to be used for measuring the study.\n14. Stop the idle flow on the Evosep by right-clicking on the Evosep logo and selecting \'Cancel maintenance procedure\'.\n15. Save the sample table.\n16. Right-click somewhere on the top row of the freshly defined sample table entries and select \'upload sample conditions\' to pre-check if all LC and MS method can be loaded correctly. The status should be change to loaded.\n17. Press \'start\' and \'start sequence\'.\n\n\n## Expected Results\nTypical chromatogram shape with MS TIC (green line) increasing drastically around 4 minutes and another increase (wash-out) at 15 minutes. Higher MS2 intensities (red line) for dia-PASEF.\n\n\n## Figures\n### Figure 1: Hystar\n[Placeholder for Screenshot of reaauired Hystar settings]\n\n\n## References\n1. Skowronek, P., Wallmann, G., Wahle, M. et al. An accessible workflow for high-sensitivity proteomics using parallel accumulation–serial fragmentation (PASEF). Nat Protoc (2025). https://doi.org/10.1038/s41596-024-01104-w\n2. TODO: Refer to Hystar manual\n3. TODO: Maybe upload video which shows protocol in action\n""",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/QueueSamples_docuWrongRow_S3A1Twice.mov""",
        "recording_type": "screen_recording",
        "ground_truth_lab_notes": """# Queue and measure samples in HyStar\n\n\n## Aim\nQueuing samples in HyStar for LC-MS measurement.\n\n\n## Materials\n### Software\nHyStar 6.0\n\n\n## Procedure\n*Timing: 2 minutes*\n\n1. Verified that Evotips are placed on top of Evosep. 5 ng HeLa Evotips at S1 from A1 to A6 and blanks at S3 from A1 to A6.\n2. Verified that the calibration of the TIMS device in TimsControl is valid.\n3. In Hystar, navigated to the 'Acquisition' tab.\n4. Chose an already existing sample table by pressing the arrow down button when hovering over the sample table name in the left sample table column.\n5. Copied already existing sample table entries to modify them.\n6. Adjusted the sample ID so that it followed this pattern: currentDate_massSpec_user_sampleType_projectID_sampleName.\n7. For performance evaluation of the LC-MS system, queued one blank, three dda-PASEF runs, three dia-PASEF runs and ended with another blank.\n8. Verified the column autocompletion settings by right-clicking on a field in the column 'vial' and selected 'Configure'. Values were set to autocomplete from A1-A12 indicated by arrows pointing to right. Ensured that the tray type was set to 'Evosep' and slots 1-6 were designated as '96Evotip'. Pressed 'OK'.\n9. ❌ **Error:** Tried to matched the Evotip position with the sample's location in the Evotip box.\nHowever, the researcher chose the wrong row to queue the HeLa samples. They selected S1 from B1 to B6 but the samples are at S1 from A1 to A6. Additionally S3 A1 is twice in sample table but every position can only be defined once within a queue.\n10. Verified 'path' folder for storing the raw files.\n11. Verified separation method.\n12. Verified that injection method is set to 'standard'.\n13. At 'MS method', adjusted dda-PASEF and dia-PASEF maintenance methods according to sample ID.\n14. ❌ **Omitted:** Missed to stop the idle flow on the Evosep by right-clicking on the Evosep logo and select 'Cancel maintenance procedure'.\n15. Saved the sample table.\n16. Right-clicked on the top row of the freshly defined sample table entries and selected 'upload sample conditions' to pre-check if all LC and MS methods could be loaded correctly. The status changed to loaded.\n17. Pressed 'start' and 'start sequence'.\n\n\n## Results\nChromatogram is not shown in video.\n""",
        "error_dict": """[
  {"Step": 1, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 2, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 3, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 4, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 5, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 6, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 7, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 8, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 9, "Benchmark": "Error", "Class": "Error", "Skill": "SpatialResolution"},
  {"Step": 10, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 11, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 12, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 13, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 14, "Benchmark": "Error", "Class": "Omitted", "Skill": "ProteomicsKnowledge"},
  {"Step": 15, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 16, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 17, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"}
]""",
        "comments": "S3 A1 twice in sample table, S1 from B1 to B6 queued for samples. However Evotips with samples are at S1 from A1 to A6, Omitted step 12",
    },
    {
        "eval_set_name": "QueueSamples_docuNoBlankNoSampleIDWrongMSmethod",
        "protocol": """# Queue and measure samples in HyStar\n\n\n## Abstract\nThis protocol explains how to queue sample in HyStar for LC-MS measurement.\n\n\n## Materials\n\n### Software\n| Software | Version | Purpose | Source/Link |\n|----------|---------|---------|------------|\n| HyStar | 6.0 | Controls LC and MS | On every instrument computer |\n\n\n## Procedure\n*Estimated timing: 6 minutes*\n\n1. Verify the Evotips were placed on top of the Evosep LC system. In this example, 5 ng HeLa Evotips were placed at S1 from A1 to A6 and blanks at S3 from A1 to A6.\n    Note: Blank Evotis can be unused and dry Evotips.\n2. Verify that the TIMS device in TimsControl is calibrated. The TIMS device should be calibrated each time before you start a sample queue.\n3. In Hystar, navigate to the \'Acquisition\' tab.\n4. Either select \'New\', and subsequently choose \'LC-MS sample table\' (Figure 1) to generate a new sample table or choose an already existing sample table by pressing the arrow down button when hovering over the sample table name in the left sample table column.\n5. In both cases, copy already existing sample table entries to modify them.\n6. Adjust the sample ID so that it follows this pattern: currentDate_massSpec_user_sampleType_projectID_ sampleName. Typical examples for sampleType: "SA_blank", "MA_HeLa", "DIAMA_HeLa".\n7. For performance evaluation of the LC-MS system, we recommend to queue one blank, three dda-PASEF runs, three dia-PASEF runs and ending with another blank.\n    Critical step: Always start the queue with a sacrificial Evotip such as a blank tip as the first run might have altered chromatographic conditions that could introduce technical errors to your measurements.\n8. Verify the column autocompletion settings with right-click on a field in the column \'vial\' such as S1-A1 in Figure 1 and select \'Configure\'. The arrows allow one to define the direction in which the vial positions on the 96-well are autocompleted when dragging values similar to Excel in the sample table. Decide whether the values should increase from A1-A12 indicated by arrows pointing to right. Ensure that the tray type is set to \'Evosep\' and slots 1-6 are designated as \'96Evotip\'. Press \'OK\'.\n9. Match the Evotip position with the sample\'s location in the Evotip box: From S1 from A1 to A6 and blanks at S3 from A1 to A6. To do this, press the arrow next to the value in the \'vial\' column (Figure 1). Select the position where the first Evotip is placed, for instance S1 A1. You can then either specify all remaining positions automatically by dragging the values (similar to Excel\'s auto-fill function) or specify each position individually.\n10. Specify a \'path\' folder for storing the raw files.\n11. Choose an existing separation method or create a new one. To create a new separation method, right-click on the separation method field, select \'new method\' followed by \'edit method\'. Choose the method type listed under \'name\' and then press \'OK\'. Specify its name and save it.\n12. Set the injection method to \'standard\'.\n13. At \'MS method\', load either dda-PASEF and dia-PASEF maintenance methods to check the LC-MS performance or the method intended to be used for measuring the study.\n14. Stop the idle flow on the Evosep by right-clicking on the Evosep logo and selecting \'Cancel maintenance procedure\'.\n15. Save the sample table.\n16. Right-click somewhere on the top row of the freshly defined sample table entries and select \'upload sample conditions\' to pre-check if all LC and MS method can be loaded correctly. The status should be change to loaded.\n17. Press \'start\' and \'start sequence\'.\n\n\n## Expected Results\nTypical chromatogram shape with MS TIC (green line) increasing drastically around 4 minutes and another increase (wash-out) at 15 minutes. Higher MS2 intensities (red line) for dia-PASEF.\n\n\n## Figures\n### Figure 1: Hystar\n[Placeholder for Screenshot of reaauired Hystar settings]\n\n\n## References\n1. Skowronek, P., Wallmann, G., Wahle, M. et al. An accessible workflow for high-sensitivity proteomics using parallel accumulation–serial fragmentation (PASEF). Nat Protoc (2025). https://doi.org/10.1038/s41596-024-01104-w\n2. TODO: Refer to Hystar manual\n3. TODO: Maybe upload video which shows protocol in action\n""",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/QueueSamples_docuNoBlankNoSampleIDWrongMSmethod.mov""",
        "recording_type": "screen_recording",
        "ground_truth_lab_notes": """# Queue and measure samples in HyStar\n\n\n## Aim\nQueuing samples in HyStar for LC-MS measurement.\n\n\n## Materials\n\n### Software\nHyStar 6.0\n\n\n## Procedure\n*Timing: 2 minutes*\n\n1. Verified that Evotips are placed on top of Evosep. 5 ng HeLa Evotips at S1 from A1 to A6 and blanks at S3 from A1 to A6.\n2. Verified that the calibration of the TIMS device in TimsControl is valid.\n3. In Hystar, navigated to the 'Acquisition' tab.\n4. Chose an already existing sample table by pressing the arrow down button when hovering over the sample table name in the left sample table column.\n5. Copied already existing sample table entries to modify them.\n7. ❌ **Error:** Queued three dda-PASEF runs, three dia-PASEF runs, but missed to queue a blank at the start and end of queue as recommended.\n8. Verified the column autocompletion settings by right-clicking on a field in the column 'vial' and selected 'Configure'. Values were set to autocomplete from A1-A12 indicated by arrows pointing to right. Ensured that the tray type was set to 'Evosep' and slots 1-6 were designated as '96Evotip'. Pressed 'OK'.\n9. Matched the Evotip position with the sample's location in the Evotip box: S1 from A1 to A6 and blanks at S3 from A1 to A6.\n13. ⚠️ **Deviation: Altered step order** & ❌ **Error:** At 'MS method', set dda-PASEF maintenance method for all samples. However, the last three samples would have to be linked to a dia-PASEF maintenance method.\n6. ⚠️ **Deviation: Altered step order** & ❌ **Error:** Adjusted the sample ID so that it followed this pattern: currentDate_massSpec_user_sampleType_sampleName, but missed to include projectID.\n10. Verified 'path' folder for storing the raw files.\n11. Verified separation method.\n12. Verified that injection method is set to 'standard'.\n15. ⚠️ **Deviation: Altered step order** Saved the sample table.\n14. Stopped the idle flow on the Evosep by right-clicking on the Evosep logo and selecting 'Cancel maintenance procedure'.\n16. ❌ **Omitted:** Missed to right-click on the top row of the freshly defined sample table entries and select 'upload sample conditions' to pre-check if all LC and MS methods could be loaded correctly. The status did not changed to loaded.\n17. Pressed 'start' and 'start sequence'.\n\n\n## Results\nChromatogram is not shown in video.\n""",
        "error_dict": """[
{"Step": 1, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 2, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 3, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 4, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 5, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 6, "Benchmark": "Error", "Class": "Deviation & Error", "Skill": "SpatialResolution"},
{"Step": 7, "Benchmark": "Error", "Class": "Error", "Skill": "SpatialResolution"},
{"Step": 8, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 9, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 10, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 11, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 12, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 13, "Benchmark": "Error", "Class": "Deviation & Error", "Skill": "SpatialResolution"},
{"Step": 14, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 15, "Benchmark": "Error", "Class": "Deviation", "Skill": "ProteomicsKnowledge"},
{"Step": 16, "Benchmark": "Error", "Class": "Omitted", "Skill": "ProteomicsKnowledge"},
{"Step": 17, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"}
]""",
        "comments": "No blanks queued, SampleID not included at sample name, Wrong MS method for last three samples, changed order, omitted upload sample conditions",
    },
    {
        "eval_set_name": "QueueSamples_docuEvotipNotAtPositionNoSampleID",
        "protocol": """# Queue and measure samples in HyStar\n\n\n## Abstract\nThis protocol explains how to queue sample in HyStar for LC-MS measurement.\n\n\n## Materials\n\n### Software\n| Software | Version | Purpose | Source/Link |\n|----------|---------|---------|------------|\n| HyStar | 6.0 | Controls LC and MS | On every instrument computer |\n\n\n## Procedure\n*Estimated timing: 6 minutes*\n\n1. Verify the Evotips were placed on top of the Evosep LC system. In this example, 5 ng HeLa Evotips were placed at S1 from A1 to A6 and blanks at S3 from A1 to A6.\n    Note: Blank Evotis can be unused and dry Evotips.\n2. Verify that the TIMS device in TimsControl is calibrated. The TIMS device should be calibrated each time before you start a sample queue.\n3. In Hystar, navigate to the \'Acquisition\' tab.\n4. Either select \'New\', and subsequently choose \'LC-MS sample table\' (Figure 1) to generate a new sample table or choose an already existing sample table by pressing the arrow down button when hovering over the sample table name in the left sample table column.\n5. In both cases, copy already existing sample table entries to modify them.\n6. Adjust the sample ID so that it follows this pattern: currentDate_massSpec_user_sampleType_projectID_ sampleName. Typical examples for sampleType: "SA_blank", "MA_HeLa", "DIAMA_HeLa".\n7. For performance evaluation of the LC-MS system, we recommend to queue one blank, three dda-PASEF runs, three dia-PASEF runs and ending with another blank.\n    Critical step: Always start the queue with a sacrificial Evotip such as a blank tip as the first run might have altered chromatographic conditions that could introduce technical errors to your measurements.\n8. Verify the column autocompletion settings with right-click on a field in the column \'vial\' such as S1-A1 in Figure 1 and select \'Configure\'. The arrows allow one to define the direction in which the vial positions on the 96-well are autocompleted when dragging values similar to Excel in the sample table. Decide whether the values should increase from A1-A12 indicated by arrows pointing to right. Ensure that the tray type is set to \'Evosep\' and slots 1-6 are designated as \'96Evotip\'. Press \'OK\'.\n9. Match the Evotip position with the sample\'s location in the Evotip box: From S1 from A1 to A6 and blanks at S3 from A1 to A6. To do this, press the arrow next to the value in the \'vial\' column (Figure 1). Select the position where the first Evotip is placed, for instance S1 A1. You can then either specify all remaining positions automatically by dragging the values (similar to Excel\'s auto-fill function) or specify each position individually.\n10. Specify a \'path\' folder for storing the raw files.\n11. Choose an existing separation method or create a new one. To create a new separation method, right-click on the separation method field, select \'new method\' followed by \'edit method\'. Choose the method type listed under \'name\' and then press \'OK\'. Specify its name and save it.\n12. Set the injection method to \'standard\'.\n13. At \'MS method\', load either dda-PASEF and dia-PASEF maintenance methods to check the LC-MS performance or the method intended to be used for measuring the study.\n14. Stop the idle flow on the Evosep by right-clicking on the Evosep logo and selecting \'Cancel maintenance procedure\'.\n15. Save the sample table.\n16. Right-click somewhere on the top row of the freshly defined sample table entries and select \'upload sample conditions\' to pre-check if all LC and MS method can be loaded correctly. The status should be change to loaded.\n17. Press \'start\' and \'start sequence\'.\n\n\n## Expected Results\nTypical chromatogram shape with MS TIC (green line) increasing drastically around 4 minutes and another increase (wash-out) at 15 minutes. Higher MS2 intensities (red line) for dia-PASEF.\n\n\n## Figures\n### Figure 1: Hystar\n[Placeholder for Screenshot of reaauired Hystar settings]\n\n\n## References\n1. Skowronek, P., Wallmann, G., Wahle, M. et al. An accessible workflow for high-sensitivity proteomics using parallel accumulation–serial fragmentation (PASEF). Nat Protoc (2025). https://doi.org/10.1038/s41596-024-01104-w\n2. TODO: Refer to Hystar manual\n3. TODO: Maybe upload video which shows protocol in action\n""",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/QueueSamples_docuEvotipNotAtPositionNoSampleID.mov""",
        "recording_type": "screen_recording",
        "ground_truth_lab_notes": """# Queue and measure samples in HyStar\n\n\n## Aim\nQueuing samples in HyStar for LC-MS measurement.\n\n\n## Materials\n\n### Software\nHyStar 6.0\n\n\n## Procedure\n*Timing: 2 minutes*\n\n1. Verified that Evotips are placed on top of Evosep. 5 ng HeLa Evotips at S1 from A1 to A6 and blanks at S3 from A1 to A6.\n2. ❌ **Omitted:** Did not mention if the TIMS device was calibrated in TimsControl.\n3. In Hystar, navigated to the 'Acquisition' tab.\n4. Chose an already existing sample table by pressing the arrow down button when hovering over the sample table name in the left sample table column.\n5. Copied already existing sample table entries to modify them.\n6. ❌ **Error:** Adjusted the sample ID so that it followed this pattern: currentDate_massSpec_user_sampleType_projectID_sampleName, but missed to include projectID.\n7. For performance evaluation of the LC-MS system, queued one blank, three dda-PASEF runs, three dia-PASEF runs and ended with another blank.\n8. Verified the column autocompletion settings by right-clicking on a field in the column 'vial' and selected 'Configure'. Values were set to autocomplete from A1-A12 indicated by arrows pointing to right. Ensured that the tray type was set to 'Evosep' and slots 1-6 were designated as '96Evotip'. Pressed 'OK'.\n9. ❌ **Error:** Matched the Evotip position with the sample's location in the Evotip box: S1 from A1 to A7 and blanks at S3 from A1 to A6. However, there is no Evotip at S1 A7.\n10. Verified 'path' folder for storing the raw files.\n11. Verified separation method.\n12. Verified that injection method is set to 'standard'.\n13. At 'MS method', adjusted dda-PASEF and dia-PASEF maintenance methods according to sample ID.\n14. ❌ **Omitted:** Missed to stop the idle flow on the Evosep by right-clicking on the Evosep logo and selecting 'Cancel maintenance procedure'.\n15. Saved the sample table.\n16. Right-clicked on the top row of the freshly defined sample table entries and selected 'upload sample conditions' to pre-check if all LC and MS methods could be loaded correctly. The status changed to loaded.\n17. Pressed 'start' and 'start sequence'.\n\n\n## Results\nChromatogram is not shown in video.\n""",
        "error_dict": """[
  {"Step": 1, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 2, "Benchmark": "Error", "Class": "Omitted", "Skill": "Fast"},
  {"Step": 3, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 4, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 5, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 6, "Benchmark": "Error", "Class": "Error", "Skill": "SpatialResolution"},
  {"Step": 7, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 8, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 9, "Benchmark": "Error", "Class": "Error", "Skill": "SpatialResolution"},
  {"Step": 10, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 11, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 12, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 13, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 14, "Benchmark": "Error", "Class": "Omitted", "Skill": "ProteomicsKnowledge"},
  {"Step": 15, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 16, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 17, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"}
]""",
        "comments": "Missed to include projectID, Queued samples for S1 A7. There is no Evotip at this position, omitted step 12",
    },
    {
        "eval_set_name": "QueueSamples_docuDiffSampleNameAndEvotipNotAtPosition",
        "protocol": """# Queue and measure samples in HyStar\n\n\n## Abstract\nThis protocol explains how to queue sample in HyStar for LC-MS measurement.\n\n\n## Materials\n\n### Software\n| Software | Version | Purpose | Source/Link |\n|----------|---------|---------|------------|\n| HyStar | 6.0 | Controls LC and MS | On every instrument computer |\n\n\n## Procedure\n*Estimated timing: 6 minutes*\n\n1. Verify the Evotips were placed on top of the Evosep LC system. In this example, 5 ng HeLa Evotips were placed at S1 from A1 to A6 and blanks at S3 from A1 to A6.\n    Note: Blank Evotis can be unused and dry Evotips.\n2. Verify that the TIMS device in TimsControl is calibrated. The TIMS device should be calibrated each time before you start a sample queue.\n3. In Hystar, navigate to the \'Acquisition\' tab.\n4. Either select \'New\', and subsequently choose \'LC-MS sample table\' (Figure 1) to generate a new sample table or choose an already existing sample table by pressing the arrow down button when hovering over the sample table name in the left sample table column.\n5. In both cases, copy already existing sample table entries to modify them.\n6. Adjust the sample ID so that it follows this pattern: currentDate_massSpec_user_sampleType_projectID_ sampleName. Typical examples for sampleType: "SA_blank", "MA_HeLa", "DIAMA_HeLa".\n7. For performance evaluation of the LC-MS system, we recommend to queue one blank, three dda-PASEF runs, three dia-PASEF runs and ending with another blank.\n    Critical step: Always start the queue with a sacrificial Evotip such as a blank tip as the first run might have altered chromatographic conditions that could introduce technical errors to your measurements.\n8. Verify the column autocompletion settings with right-click on a field in the column \'vial\' such as S1-A1 in Figure 1 and select \'Configure\'. The arrows allow one to define the direction in which the vial positions on the 96-well are autocompleted when dragging values similar to Excel in the sample table. Decide whether the values should increase from A1-A12 indicated by arrows pointing to right. Ensure that the tray type is set to \'Evosep\' and slots 1-6 are designated as \'96Evotip\'. Press \'OK\'.\n9. Match the Evotip position with the sample\'s location in the Evotip box: From S1 from A1 to A6 and blanks at S3 from A1 to A6. To do this, press the arrow next to the value in the \'vial\' column (Figure 1). Select the position where the first Evotip is placed, for instance S1 A1. You can then either specify all remaining positions automatically by dragging the values (similar to Excel\'s auto-fill function) or specify each position individually.\n10. Specify a \'path\' folder for storing the raw files.\n11. Choose an existing separation method or create a new one. To create a new separation method, right-click on the separation method field, select \'new method\' followed by \'edit method\'. Choose the method type listed under \'name\' and then press \'OK\'. Specify its name and save it.\n12. Set the injection method to \'standard\'.\n13. At \'MS method\', load either dda-PASEF and dia-PASEF maintenance methods to check the LC-MS performance or the method intended to be used for measuring the study.\n14. Stop the idle flow on the Evosep by right-clicking on the Evosep logo and selecting \'Cancel maintenance procedure\'.\n15. Save the sample table.\n16. Right-click somewhere on the top row of the freshly defined sample table entries and select \'upload sample conditions\' to pre-check if all LC and MS method can be loaded correctly. The status should be change to loaded.\n17. Press \'start\' and \'start sequence\'.\n\n\n## Expected Results\nTypical chromatogram shape with MS TIC (green line) increasing drastically around 4 minutes and another increase (wash-out) at 15 minutes. Higher MS2 intensities (red line) for dia-PASEF.\n\n\n## Figures\n### Figure 1: Hystar\n[Placeholder for Screenshot of reaauired Hystar settings]\n\n\n## References\n1. Skowronek, P., Wallmann, G., Wahle, M. et al. An accessible workflow for high-sensitivity proteomics using parallel accumulation–serial fragmentation (PASEF). Nat Protoc (2025). https://doi.org/10.1038/s41596-024-01104-w\n2. TODO: Refer to Hystar manual\n3. TODO: Maybe upload video which shows protocol in action\n""",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/QueueSamples_docuDiffSampleNameAndEvotipNotAtPosition.mov""",
        "recording_type": "screen_recording",
        "ground_truth_lab_notes": """# Queue and measure samples in HyStar\n\n\n## Aim\nQueuing samples in HyStar for LC-MS measurement.\n\n\n## Materials\n\n### Software\nHyStar 6.0\n\n\n## Procedure\n*Timing: 2 minutes*\n\n1. Verified that Evotips are placed on top of Evosep. 5 ng HeLa Evotips at S1 from A1 to A6 and blanks at S3 from A1 to A6.\n2. Verified that the calibration of the TIMS device in TimsControl is valid.\n3. In Hystar, navigated to the 'Acquisition' tab.\n4. Chose an already existing sample table by pressing the arrow down button when hovering over the sample table name in the left sample table column.\n5. Copied already existing sample table entries to modify them.\n. Adjusted the sample ID so that it followed this pattern: currentDate_massSpec_user_sampleType_projectID_sampleName.\n7. ❌ **Error:** For measuring a study, queued one blank, two dia-PASEF runs, but missed to queue blank at the end of the queue.\n8. ❌ **Omitted:** Missed to verify the column autocompletion settings.\n9. ❌ **Error:** Did not match the Evotip position with the sample's location in the Evotip box.\n10. Verified 'path' folder for storing the raw files.\n11. Verified separation method.\n12. Verified that injection method is set to 'standard'.\n13. At 'MS method', adjusted dda-PASEF and dia-PASEF maintenance methods according to sample ID.\n14. ❌ **Omitted:** Stopped the idle flow on the Evosep by right-clicking on the Evosep logo and selecting 'Cancel maintenance procedure'.\n15. Saved the sample table.\n16. Right-clicked on the top row of the freshly defined sample table entries and selected 'upload sample conditions' to pre-check if all LC and MS methods could be loaded correctly. The status changed to loaded.\n17. Pressed 'start' and 'start sequence'.\n\n\n## Results\nChromatogram is not shown in video.\n""",
        "error_dict": """[
  {"Step": 1, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 2, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 3, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 4, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 5, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 6, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 7, "Benchmark": "Error", "Class": "Error", "Skill": "SpatialResolution"},
  {"Step": 8, "Benchmark": "Error", "Class": "Omitted", "Skill": "ProteomicsKnowledge"},
  {"Step": 9, "Benchmark": "Error", "Class": "Error", "Skill": "SpatialResolution"},
  {"Step": 10, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 11, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 12, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 13, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 14, "Benchmark": "Error", "Class": "Omitted", "Skill": "ProteomicsKnowledge"},
  {"Step": 15, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 16, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 17, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"}
]""",
        "comments": "study samples and not performance evotips, no blank queued for the end of the queue, Queued sample for S1 H2. There is no Evotip at this position, Omitted step 6 and 12",
    },
    {
        "eval_set_name": "Pipette_docu_WrongRowB_B10Twice_B11LeftOut",
        "protocol": "# Dispensing Protocol\n\n\n## Abstract\nThis protocol describes how to pipette 20 uL buffer A in a PCR Plate from A1 to A12.\n\n\n## Materials\n\n### Equipment\n- PCR Plate\n- Pipette\n\n### Reagents\n- Formic acid (FA)\n  ! CAUTION: This liquid may be corrosive. It is harmful and can cause damage if direct contact occurs.\n\n### Reagent setup\n- Buffer A: Consists of 0.1% (vol/vol) FA. The buffers are stable for at least 6 months at room temperature as long as they are protected from sunlight.\n\n\n## Procedure\n*Estimated timing: 1 minute*\n\n1. Open the bottle of Buffer A.\n2. Adjust the pipette to 20 μL setting.\n3. Attach a pipette tip to the pipette.\n4. Pipette 20 μL of Buffer A from the bottle into well A1.\n5. Continue dispensing 20 μL of Buffer A into each consecutive well from A2 to A12.\n6. Discard the used pipette tip in the appropriate waste container.\n\n\n## Expected Results\nApproximatly 20 uL liquid in A1 to A12 in a PCR Plate.\n",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/Pipette_docu_WrongRowB_B10Twice_B11LeftOut.MP4""",
        "recording_type": "camera",
        "ground_truth_lab_notes": """# Dispensing Protocol\n\n\n## Aim\nPipette 20 uL buffer A in a PCR Plate from A1 to A12.\n\n\n## Materials\n\n### Equipment\n- PCR Plate\n- Pipette\n\n### Reagent setup\n- Buffer A: Consists of 0.1% (vol/vol) FA.\n\n\n## Procedure\n*Timing: 1 minute*\n\n2. ⚠️ **Deviation: Altered step order** Adjusted the pipette to 20 μL setting.\n1. Opened the bottle of Buffer A.\n3. Attached a pipette tip to the pipette.\n4. ❌ **Error:** Pipetted 20 μL of Buffer A from the bottle into well B1 instead of A1\n5. ❌ **Error:** Continueed dispensing 20 μL of Buffer A into consecutive wells from B2 to B10 and into B12. However it should have been pipetted in to row A. Additionally, they pipetted twice into B10, left out B11.\n6. Discarded the used pipette tip in the appropriate waste container.\n\n\n## Result\nApproximatly 20 uL liquid in B1 to B9, 40 uL in B10, 20 uL in B12.\n""",
        "error_dict": """[
  {"Step": 1, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 2, "Benchmark": "Error", "Class": "Deviation", "Skill": "ProteomicsKnowledge"},
  {"Step": 3, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 4, "Benchmark": "Error", "Class": "Error", "Skill": "SpatialOrientation"},
  {"Step": 5, "Benchmark": "Error", "Class": "Error", "Skill": "SpatialOrientation"},
  {"Step": 6, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"}
]""",
        "comments": "Pipetted into row B instead of row A, Pipetted twice into B10, left out B11",
    },
    {
        "eval_set_name": "Pipette_docuFirstTwoWrongVolumneANDA2Twice",  # pragma: allowlist secret
        "protocol": "# Dispensing Protocol\n\n\n## Abstract\nThis protocol describes how to pipette 20 uL buffer A in a PCR Plate from A1 to A12.\n\n\n## Materials\n\n### Equipment\n- PCR Plate\n- Pipette\n\n### Reagents\n- Formic acid (FA)\n  ! CAUTION: This liquid may be corrosive. It is harmful and can cause damage if direct contact occurs.\n\n### Reagent setup\n- Buffer A: Consists of 0.1% (vol/vol) FA. The buffers are stable for at least 6 months at room temperature as long as they are protected from sunlight.\n\n\n## Procedure\n*Estimated timing: 1 minute*\n\n1. Open the bottle of Buffer A.\n2. Adjust the pipette to 20 μL setting.\n3. Attach a pipette tip to the pipette.\n4. Pipette 20 μL of Buffer A from the bottle into well A1.\n5. Continue dispensing 20 μL of Buffer A into each consecutive well from A2 to A12.\n6. Discard the used pipette tip in the appropriate waste container.\n\n\n## Expected Results\nApproximatly 20 uL liquid in A1 to A12 in a PCR Plate.\n",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/Pipette_docuFirstTwoWrongVolumneANDA2Twice.MP4""",
        "recording_type": "camera",
        "ground_truth_lab_notes": """# Dispensing Protocol\n\n\n## Aim\nPipette 20 uL buffer A in a PCR Plate from A1 to A12.\n\n\n## Materials\n\n### Equipment\n- PCR Plate\n- Pipette\n\n### Reagent setup\n- Buffer A: Consists of 0.1% (vol/vol) FA.\n\n\n## Procedure\n*Timing: 1 minute*\n\n1. Opened the bottle of Buffer A.\n2. ❌ **Omitted:** Did not adjust the pipette to 20 μL setting.\n3. Attached a pipette tip to the pipette.\n4. Pipetted 20 μL of Buffer A from the bottle into well A1 and A2.\n4.1 ➕ **Added:** Adjusted the pipette to 20 μL setting.\n5. ❌ **Error:** Continueed dispensing 20 μL of Buffer A into consecutive wells from A2 to A12 thereby pipetting twice into A2.\n6. Discarded the used pipette tip in the appropriate waste container.\n6.1 ➕ **Added:** Closed the bottle of Buffer A.\n\n\n## Result\n- Approximatly 20 uL liquid in A1, 40 uL in A2 and 20 uL from A3 to A12.\n""",
        "error_dict": """[
  {"Step": 1, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 2, "Benchmark": "Error", "Class": "Omitted", "Skill": "ProteomicsKnowledge"},
  {"Step": 3, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 4, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 4.1, "Benchmark": "Error", "Class": "Addition", "Skill": "ProteomicsKnowledge"},
  {"Step": 5, "Benchmark": "Error", "Class": "Error", "Skill": "SpatialOrientation"},
  {"Step": 6, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 6.1, "Benchmark": "Error", "Class": "Addition", "Skill": "GeneralKnowledge"}
]""",
        "comments": "Pipetted A1 and A2 with wrong volume, Pipetted twice into A2",
    },
    {
        "eval_set_name": "Pipette_ducoBufferBInsteadBufferA",
        "protocol": "# Dispensing Protocol\n\n\n## Abstract\nThis protocol describes how to pipette 20 uL buffer A in a PCR Plate from A1 to A12.\n\n\n## Materials\n\n### Equipment\n- PCR Plate\n- Pipette\n\n### Reagents\n- Formic acid (FA)\n  ! CAUTION: This liquid may be corrosive. It is harmful and can cause damage if direct contact occurs.\n\n### Reagent setup\n- Buffer A: Consists of 0.1% (vol/vol) FA. The buffers are stable for at least 6 months at room temperature as long as they are protected from sunlight.\n\n\n## Procedure\n*Estimated timing: 1 minute*\n\n1. Open the bottle of Buffer A.\n2. Adjust the pipette to 20 μL setting.\n3. Attach a pipette tip to the pipette.\n4. Pipette 20 μL of Buffer A from the bottle into well A1.\n5. Continue dispensing 20 μL of Buffer A into each consecutive well from A2 to A12.\n6. Discard the used pipette tip in the appropriate waste container.\n\n\n## Expected Results\nApproximatly 20 uL liquid in A1 to A12 in a PCR Plate.\n",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/Pipette_ducoBufferBInsteadBufferA.MP4""",
        "recording_type": "camera",
        "ground_truth_lab_notes": """# Dispensing Protocol\n\n\n## Aim\nPipette 20 uL buffer A in a PCR Plate from A1 to A2.\n\n\n## Materials\n\n### Equipment\n- PCR Plate\n- Pipette\n\n### Reagent setup\n- Buffer A: Consists of 0.1% (vol/vol) FA.\n\n\n## Procedure\n*Timing: 1 minute*\n\n2. ⚠️ **Deviation: Altered step order** Adjusted the pipette to 20 μL setting.\n1. ❌ **Error:** Opened the bottle of Buffer B, but should have used buffer A instead.\n3. Attached a pipette tip to the pipette.\n4. Pipetted 20 μL of Buffer A from the bottle into well A1.\n5. ❌ **Error:** Continueed dispensing 20 μL of Buffer A into consecutive wells from A3 to A12, but left out to pipetted A2.\n6. Discarded the used pipette tip in the appropriate waste container.\n6.1 ➕ **Added:** Closed the bottle of Buffer A.\n\n\n## Result\nApproximatly 20 uL liquid in A1 to A12 in a PCR Plate.\n""",
        "error_dict": """[
  {"Step": 1, "Benchmark": "Error", "Class": "Error", "Skill": "SpatialResolution"},
  {"Step": 2, "Benchmark": "Error", "Class": "Deviation", "Skill": "ProteomicsKnowledge"},
  {"Step": 3, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 4, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 5, "Benchmark": "Error", "Class": "Error", "Skill": "SpatialResolution"},
  {"Step": 6, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 6.1, "Benchmark": "Error", "Class": "Addition", "Skill": "GeneralKnowledge"}
]""",
        "comments": "wrong solvents (EvoA ↔︎ EvoB), Left out to pipette A2",
    },
    {
        "eval_set_name": "Dilute_docuWrongVolume_PipettTipNotChanged",
        "protocol": "# Dispensing Protocol\n\n## Abstract\nThis protocol describes how to pipette first 400 uL buffer A and then 100 uL buffer B in a Eppendorf tube.\n\n\n## Materials\n\n### Equipment\n- Eppendorf tube\n- Pipette\n\n### Reagents\n- Formic acid (FA)\n  - ! CAUTION: This liquid may be corrosive. It is harmful and can cause damage if direct contact occurs.\n- Acetonitrile\n  - ! CAUTION: This liquid is highly flammable and can be harmful if contact occurs.\n\n### Reagent setup\n- Buffer A: Consists of 0.1% (vol/vol) FA.\n- Buffer B: Consists of 0.1% (vol/vol) FA/99.9% (vol/vol) acetonitrile.\n\n\n## Procedure\n*Estimated timing: 1 minute*\n\n1. Open the bottles of Buffer A and Buffer B.\n2. Adjust the pipette to 400 μL setting.\n3. Attach a pipette tip to the pipette.\n4. Pipette 400 μL of Buffer A from the bottle into the Eppendorf tube.\n5. Adjust the pipette to 100 μL setting.\n6. Discard the used pipette tip in the appropriate waste container.\n7. Attach a pipette tip to the pipette.\n8. Pipette 100 μL of Buffer B from the bottle into the Eppendorf tube.\n9. Discard the used pipette tip in the appropriate waste container.\n10. Mix the solution with a vortexer.\n11. Lable the Eppendorf tube.\n\n\n## Expected Results\nApproximatly 500 uL liquid in the tube\n",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/Dilute_docuWrongVolume_PipettTipNotChanged.MP4""",
        "recording_type": "camera",
        "ground_truth_lab_notes": """# Dispensing Protocol\n\n## Aim\nPipetting first 400 uL buffer A and then 100 uL buffer B in a Eppendorf tube.\n\n\n## Materials\n\n### Equipment\n- Eppendorf tube\n- Pipette\n\n### Reagents\n- Formic acid (FA)\n  - ! CAUTION: This liquid may be corrosive. It is harmful and can cause damage if direct contact occurs.\n- Acetonitrile\n  - ! CAUTION: This liquid is highly flammable and can be harmful if contact occurs.\n\n### Reagent setup\n- Buffer A: Consists of 0.1% (vol/vol) FA.\n- Buffer B: Consists of 0.1% (vol/vol) FA/99.9% (vol/vol) acetonitrile.\n\n\n## Procedure\n*Timing: 1 minute*\n\n2. ⚠️ **Deviation: Altered step order** & ❌ **Error:** Adjusted the pipette volume to 430 μL instead of 400 uL.\n1. Opened the bottles of Buffer A and Buffer B.\n3. Attached a pipette tip to the pipette.\n4. Pipetted 430 μL of Buffer A from the bottle into the Eppendorf tube.\n5. Adjusted the pipette to 100 μL setting.\n6. ❌ **Omitted:** Did not discard the used pipette tip in the appropriate waste container.\n7. ❌ **Omitted:** Did not attach a new pipette tip to the pipette.\n8. Pipetted 100 μL of Buffer B from the bottle into the Eppendorf tube containing Buffer A.\n9. Discarded the used pipette tip in the appropriate waste container.\n10. Mixed the solution with a vortexer.\n11. Labeled the Eppendorf tube.\n11.1 ➕ **Added:** Closed the bottles of Buffer A and Buffer B.\n11.2 ➕ **Added:** Tidied up the bench.\n\n\n## Results\n- 530 uL liquid are in the tube\n""",
        "error_dict": """[
  {"Step": 1, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 2, "Benchmark": "Error", "Class": "Deviation & Error", "Skill": "SpatialResolution"},
  {"Step": 3, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 4, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 5, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 6, "Benchmark": "Error", "Class": "Omitted", "Skill": "GeneralKnowledge"},
  {"Step": 7, "Benchmark": "Error", "Class": "Omitted", "Skill": "GeneralKnowledge"},
  {"Step": 8, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 9, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 10, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 11, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 11.1, "Benchmark": "Error", "Class": "Addition", "Skill": "GeneralKnowledge"},
  {"Step": 11.2, "Benchmark": "Error", "Class": "Addition", "Skill": "GeneralKnowledge"}
]""",
        "comments": "Should have been exactly 400 uL, omitted step 6 & 7: Discarding the used pipette tip, Attaching a new pipette tip, Added: Closing the bottles of Buffer A and Buffer B, Tieing up the bench",
    },
    {
        "eval_set_name": "Dilute_docuWrongSolvent",
        "protocol": "# Dispensing Protocol\n\n## Abstract\nThis protocol describes how to pipette first 400 uL buffer A and then 100 uL buffer B in a Eppendorf tube.\n\n\n## Materials\n\n### Equipment\n- Eppendorf tube\n- Pipette\n\n### Reagents\n- Formic acid (FA)\n  - ! CAUTION: This liquid may be corrosive. It is harmful and can cause damage if direct contact occurs.\n- Acetonitrile\n  - ! CAUTION: This liquid is highly flammable and can be harmful if contact occurs.\n\n### Reagent setup\n- Buffer A: Consists of 0.1% (vol/vol) FA.\n- Buffer B: Consists of 0.1% (vol/vol) FA/99.9% (vol/vol) acetonitrile.\n\n\n## Procedure\n*Estimated timing: 1 minute*\n\n1. Open the bottles of Buffer A and Buffer B.\n2. Adjust the pipette to 400 μL setting.\n3. Attach a pipette tip to the pipette.\n4. Pipette 400 μL of Buffer A from the bottle into the Eppendorf tube.\n5. Adjust the pipette to 100 μL setting.\n6. Discard the used pipette tip in the appropriate waste container.\n7. Attach a pipette tip to the pipette.\n8. Pipette 100 μL of Buffer B from the bottle into the Eppendorf tube.\n9. Discard the used pipette tip in the appropriate waste container.\n10. Mix the solution with a vortexer.\n11. Lable the Eppendorf tube.\n\n\n## Expected Results\nApproximatly 500 uL liquid in the tube\n",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/Dilute_docuWrongSolvent.MP4""",
        "recording_type": "camera",
        "ground_truth_lab_notes": """# Dispensing Protocol\n\n## Aim\nPipetting first 400 uL buffer A and then 100 uL buffer B in a Eppendorf tube.\n\n\n## Materials\n\n### Equipment\n- Eppendorf tube\n- Pipette\n\n### Reagents\n- Formic acid (FA)\n  - ! CAUTION: This liquid may be corrosive. It is harmful and can cause damage if direct contact occurs.\n- Acetonitrile\n  - ! CAUTION: This liquid is highly flammable and can be harmful if contact occurs.\n\n### Reagent setup\n- Buffer A: Consists of 0.1% (vol/vol) FA.\n- Buffer B: Consists of 0.1% (vol/vol) FA/99.9% (vol/vol) acetonitrile.\n\n\n## Procedure\n*Timing: 1 minute*\n\n1. Opened the bottles of Buffer A and Buffer B.\n2. Adjusted the pipette to 400 μL setting.\n3. Attached a pipette tip to the pipette.\n4. ❌ **Error:** Pipetted 400 μL of Buffer B from the bottle into the Eppendorf tube, but it should have been buffer A instead.\n5. Adjusted the pipette to 100 μL setting.\n6. Discarded the used pipette tip in the appropriate waste container.\n7. Attached a new pipette tip to the pipette.\n8. ❌ **Error:** Pipetted 100 μL of Buffer A from the bottle into the Eppendorf tube containing Buffer A, but it should have been buffer B instead. Additionally, there were bubbels in pipette tip.\n9. Discarded the used pipette tip in the appropriate waste container.\n10. ❌ **Omitted:** Did not mix the solution with a vortexer.\n11. ❌ **Omitted:** Did not label the Eppendorf tube.\n\n\n## Results\n500 uL liquid in the tube but with reversed solvent composition than intented\n""",
        "error_dict": """[
{"Step": 1, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 2, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 3, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 4, "Benchmark": "Error", "Class": "Error", "Skill": "SpatialResolution"},
{"Step": 5, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 6, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 7, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 8, "Benchmark": "Error", "Class": "Error", "Skill": "SpatialResolution"},
{"Step": 9, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
{"Step": 10, "Benchmark": "Error", "Class": "Omitted", "Skill": "GeneralKnowledge"},
{"Step": 11, "Benchmark": "Error", "Class": "Omitted", "Skill": "GeneralKnowledge"}
]""",
        "comments": "Should have used first buffer A and then buffer B, Omitted step 10 & 11: mixing and lebeling",
    },
    {
        "eval_set_name": "Evotip_docuWrongSpinInEvotipBox_wrongSpeed",
        "protocol": "# Dispensing Protocol\n\n\n## Abstract\nThis protocol describes how to spin Evotips.\n\n\n## Materials\n\n### Equipment\n- Evotips\n- Evotip holders wilth solvent reservior\n- Centrifuge\n\n\n## Procedure\n*Estimated timing: 1 minute*\n\n1. Equaly distribute Evotips onto two Evotip holders. For instance 12 Evotips each. Do this either manually or by using a multichannel pipette.\n2. Switch on the centrifuge.\n3. Place the Evotip holders within centrifuge holders at opposite positions.\n4. Spin the Evotips at 800g for 1 minute in a centrifuge.\n\n\n## Expected Results\nA balanced centrifuge that spins at 800g for 1 minute.\n",
        "video_path": """gs://ai-proteomics-advisor/input_for_lab_note/Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4""",
        "recording_type": "camera",
        "ground_truth_lab_notes": """# Dispensing Protocol\n\n\n## Aim\nSpinning Evotips.\n\n\n## Materials\n\n### Equipment\n- Evotips\n- Evotip holders wilth solvent reservior\n- Centrifuge\n\n\n## Procedure\n*Timing: 1 minute*\n\n1. ❌ **Error:** Equaly distributed Evotips with 12 Evotips each by using a multichannel pipette. 12 Evotips each. However, used Evotip boxes instead of Evotip holders.\n2. ❌ **Omitted:** Did not switching on the centrifuge as it was already in operation.\n3. Placed the Evotip holders within centrifuge holders at opposite positions.\n4. ❌ **Error:** Spun the Evotips at 700g for 1 minute in a centrifuge. However, they should have been spun Evotips at 800g instead.\n\n\n## Results\nEvotips were spun in a balanced centrifuge at 700g for 1 minute.\n""",
        "error_dict": """[
  {"Step": 1, "Benchmark": "Error", "Class": "Error", "Skill": "ProteomicsKnowledge"},
  {"Step": 2, "Benchmark": "Error", "Class": "Omitted", "Skill": "GeneralKnowledge"},
  {"Step": 3, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},
  {"Step": 4, "Benchmark": "Error", "Class": "Error", "Skill": "SpatialResolution"}
]""",
        "comments": "Used Evotip boxes instead of Evotip holders to spin, Should have been spun Evotips at 800g instead of 700g",
    },
]
df_benchmark_data = pd.DataFrame(list_new_rows)

df_benchmark_data.to_csv(benchmark_dataset_path, index=False)
df_benchmark_data

In [ ]:
# Loading the content of a md file
from IPython.display import Markdown


def load_md_as_text(file_path: str) -> None:
    """Load markdown file as plain text string."""
    with Path.open(file_path, encoding="utf-8") as file:
        return file.read()


md_content = load_md_as_text(
    "/Users/patriciaskowronek/Documents/proteomics_specialist/data/UltraSourceToESIsource_protocolCorrect.md"
)
md_content

In [ ]:
# Runing lab note eval pipline
from eval.eval_lab_note_generation.eval_analysis_run import EvaluationAnalyzer

output_dir = "/Users/patriciaskowronek/Documents/proteomics_specialist/eval/eval_lab_note_generation/eval_lab_note_results/result_20250803_061506"
analyzer = EvaluationAnalyzer(output_dir=output_dir)
metrics_dict = analyzer.run_complete_analysis(
    Path(output_dir) / "all_eval_sets_all_runs.json"
)
metrics_dict

In [ ]:
# Inspecting individual results from log file
import json

from IPython.display import Markdown


def load_json_basic(file_path: str) -> None:
    """Load JSON file using built-in json module."""
    with Path.open(file_path, encoding="utf-8") as file:
        return json.load(file)


file_path = "/Users/patriciaskowronek/Documents/proteomics_specialist/eval/eval_lab_note_generation/eval_lab_note_results/result_20250803_061506/eval_set_PlaceEvotips_docuCorrect_all_runs.json"
json_data = load_json_basic(file_path)
Markdown(json_data[0]["lab_notes"])

In [ ]:
pd.DataFrame(json_data[0]["df_errors"])

In [ ]:
import os
from typing import TYPE_CHECKING

from google.cloud import storage

if TYPE_CHECKING:
    from google.cloud.storage import Bucket


def list_files_in_gcs_folder(bucket: Bucket, prefix: str) -> list:
    """Lists all files in a GCS 'folder' (prefix)."""
    blobs = bucket.list_blobs(prefix=prefix)

    blob_info = []
    for blob in blobs:
        info = {
            "name": blob.name,
            "size": blob.size,
            "updated": blob.updated,
            "created": blob.time_created,
            "content_type": blob.content_type,
            "etag": blob.etag,
            "generation": blob.generation,
            "md5_hash": blob.md5_hash,
            "crc32c": blob.crc32c,
            "storage_class": blob.storage_class,
            "metadata": blob.metadata,
            "public_url": blob.public_url,
            "media_link": blob.media_link,
        }
        blob_info.append(info)

    return blob_info


bucket_name = os.getenv("GOOGLE_CLOUD_STORAGE_BUCKET")

storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)

folder_path = "input_for_lab_note/"

blob_list = list_files_in_gcs_folder(bucket, folder_path)
print(f"Found {len(blob_list)} files:")
print(blob_list)